# Introduction

In this project, we will try to see if there is a relation between the intensity of care of cardiologists and
their vertical integration status.

First, we will set our environment up. We used discovery cluster for this project.


# Setting up the environment

We'll load the needed libraries:


In [ ]:
packrat::init("/work/postresearch/Shared/Projects/Farbod/CaseMix")
#install.packages("remotes")
#remotes::install_github("ellessenne/comorbidity@0.5.3")

setwd("/work/postresearch/Shared/Projects/Farbod")
packrat::on()
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)

numcores=56
numcores_foreach=56

library(tidyverse)
library(parallel)
library(data.table)
library(fst)
library(comorbidity)
library(zeallot)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)


library(foreach)
library(doMC)
registerDoMC(cores=numcores_foreach)

Initializing packrat project in directory:
- "/work/postresearch/Shared/Projects/Farbod/CaseMix"

Initialization complete!

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘reshape’


The following object is masked from ‘package:data.table’:

    melt


The following object is masked from ‘packag

ERROR: Error in library(foreach): there is no package called ‘foreach’


# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [40]:
#diagnosis codes
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52850&ver=26 and https://www.aapc.com/codes/cpt-codes-range/93451-93533/10
angio_codes=c(93451,93452,93453,93454,93455,93456,93457,93458,93459,93460,93461,93462,93463,93464
              ,93503,93505,93530,93531,93532,93533)
#from https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleid=57326&ver=13&keyword=electrocardiogram&keywordType=starts&areaId=all&docType=NCA,CAL,NCD,MEDCAC,TA,MCD,6,3,5,1,F,P&contractOption=all&sortBy=relevance&bc=1
ecg_codes=c(93000,93005,93010,93040,93041,93042)
#from https://scct.org/page/CardiacCTCodes include CTangio
cardiac_ct_codes=c(75571,75572,75573,75574)
#from https://cardiacmri.com/tech-guide/cpt-codes-relevant-to-cardiac-mri/
cardiac_mri_codes=c(75557,75559,75561,75563,75565)
# from https://medicarepaymentandreimbursement.com/2011/07/cardiovascular-stress-testing-cpt-93015.html and https://www.aapc.com/codes/cpt-codes-range/93000-93050/
stress_test_codes=c(93015,93016,93017,93018)
#from https://www.aapc.com/codes/cpt-codes-range/93303-93356/20     includes stress echo
echocardiography_codes=c(93303,93304,93306,93307,93308,93312,93313,93314,93315,93316,93317,93318
                         ,93320,93321,93325,93350,93351,93356,93352,93355,93356)
#from https://www.aapc.com/codes/cpt-codes-range/92920-92979/ and https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=57479#:~:text=CPT%20codes%2092928%2C%2092933%2C%2092929,are%20assigned%20to%20APC%200104.    includes balloon and stent
angioplasty_codes=c(92920,92921,92924,92925,92928,92929,92933,92934,92937,92938,92941,92943,92944
                    ,92973,92974,92975,92978,92979,93571,93572,"C9600","C9601","C9602","C9603"
                    ,"C9604","C9605","C9606","C9607","C9608")
#from https://www.medaxiom.com/clientuploads/webcast_handouts/Coding_for_CABG-Open_Heart_Procedures.pdf and https://www.aapc.com/codes/cpt-codes-range/33016-33999/10    did not include 33517-33530 since these are used in conjunction with 33533-33548 and not alone, did not include 33542,33545,33548 since these are also in conjunction )aneurismectomy and vsd resection
CABG_codes=c(33510,33511,33512,33513,33514,33516,33533,33534,33535,33536)
#from http://www.icd9data.com/2015/Volume1/390-459/430-438/default.htm and https://www.icd10data.com/ICD10CM/Codes/I00-I99/I60-I69/I63-
stroke_icd_9_codes=c(43301,43311,43321,43331,43381,43391,43401,43411,43491)
office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")
IHD_icd_9_codes=c(410, 411, 412,413,414)
IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)

primary_care_specialty_codes=c("01", "08", "11", "38")
surgery_specialty_codes=c("02","04","14","19","20","24","28","33","34","40","48","77","78","85","91")

#http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes=c("401","402","403","404","405")
hypertension_icd_10_codes=c("I10","I11","I12","I13","I15","I16")

#http://www.icd9data.com/2014/Volume1/460-519/490-496/default.htm
#https://www.icd10data.com/ICD10CM/Codes/J00-J99/J40-J47
copd_icd_9_codes=c("490","491","492","493","494","495","496")
copd_icd_10_codes=c("J40","J41","J42","J43","J44","J45","J47")

#http://www.icd9data.com/2015/Volume1/240-279/270-279/278/278.htm?__hstc=93424706.cdd51240e438a5219319ce13ccb23860.1648603374124.1648603374124.1648607295327.2&__hssc=93424706.9.1648607295327&__hsfp=908776442
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E65-E68/E66-
obesity_icd_9_codes=c("2780","V853","V854")
obesity_icd_10_codes=c("E660","E669","Z683","Z684")

#http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
#https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes=c("2962","2963")
depression_icd_10_codes=c("F32","F33")

#http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes=c("250")
diabetes_icd_10_codes=c("E08","E09","E10","E11","E13")


# Reading the raw data and choosing columns

## Reading carrier, inpatient, and outpatient files

### Carrier files

In [19]:
carrier_data_2013=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2013.fst",as.data.table = T)
#carrier_data_2014=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2014.fst",as.data.table = T)
#carrier_data_2015=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2015.fst",as.data.table = T)
#carrier_data_2016=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2016.fst",as.data.table = T)
#carrier_data_2017=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2017.fst",as.data.table = T)
#carrier_data_2018=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2018.fst",as.data.table = T)
#carrier_data_2019=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2019.fst",as.data.table = T)
#carrier_data_2020=read_fst("/work/postresearch/Shared/Projects/Data_fst/carrier_data_2020.fst",as.data.table = T)

### Inpatient files

In [52]:
inpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2013.fst",as.data.table = T)
inpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2014.fst",as.data.table = T)
inpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2015.fst",as.data.table = T)
inpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2016.fst",as.data.table = T)
inpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2017.fst",as.data.table = T)
inpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2018.fst",as.data.table = T)
inpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2019.fst",as.data.table = T)
inpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/inpatient_data_2020.fst",as.data.table = T)

### Outpatient files

In [ ]:
outpatient_data_2013 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2013.fst",as.data.table = T)
outpatient_data_2014 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2014.fst",as.data.table = T)
outpatient_data_2015 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2015.fst",as.data.table = T)
outpatient_data_2016 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2016.fst",as.data.table = T)
outpatient_data_2017 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2017.fst",as.data.table = T)
outpatient_data_2018 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2018.fst",as.data.table = T)
outpatient_data_2019 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2019.fst",as.data.table = T)
outpatient_data_2020 = read_fst("/work/postresearch/Shared/Projects/Data_fst/outpatient_data_2020.fst",as.data.table = T)

In [28]:
#carrier_data_2013[DESY_SORT_KEY==100000099]
#inpatient_data_2013[DESY_SORT_KEY==100000099]
#outpatient_data_2013[DESY_SORT_KEY==100000099]

#carrier_data_2013[1:10]
#inpatient_data_2013[1:10]
#outpatient_data_2013[1:10]

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,NCH_CLM_TYPE_CD,CARR_PRFRNG_PIN_NUM,PRF_PHYSN_UPIN,PRF_PHYSN_NPI,ORG_NPI_NUM,CARR_LINE_PRVDR_TYPE_CD,PRVDR_STATE_CD,PRVDR_SPCLTY,PRTCPTNG_IND_CD,CARR_LINE_RDCD_PMT_PHYS_ASTN_C,LINE_SRVC_CNT,LINE_CMS_TYPE_SRVC_CD,LINE_PLACE_OF_SRVC_CD,CARR_LINE_PRCNG_LCLTY_CD,LINE_LAST_EXPNS_DT,HCPCS_CD,HCPCS_1ST_MDFR_CD,HCPCS_2ND_MDFR_CD,BETOS_CD,LINE_NCH_PMT_AMT,LINE_BENE_PMT_AMT,LINE_PRVDR_PMT_AMT,LINE_BENE_PTB_DDCTBL_AMT,LINE_BENE_PRMRY_PYR_CD,LINE_BENE_PRMRY_PYR_PD_AMT,LINE_COINSRNC_AMT,LINE_SBMTD_CHRG_AMT,LINE_ALOWD_CHRG_AMT,LINE_PRCSG_IND_CD,LINE_PMT_80_100_CD,LINE_SERVICE_DEDUCTIBLE,CARR_LINE_MTUS_CNT,CARR_LINE_MTUS_CD,LINE_ICD_DGNS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_HCT_HGB_RSLT_NUM,LINE_HCT_HGB_TYPE_CD,LINE_NDC_CD,CARR_LINE_CLIA_LAB_NUM,CARR_LINE_ANSTHSA_UNIT_CNT
<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<dbl>,<chr>,<lgl>,<chr>,<int>
100000099,173,1,20130104,71,49196Z,,1598867251,NA,1,10,11,2,0,1,1,11,03,20130104,99211,,,M1B,0.00,0,0.00,21.46,,0,0.00,37.00,21.46,A,0,0,1,3,42731,9,0,,NA,,0
100000099,173,2,20130104,71,49196Z,,1598867251,NA,1,10,11,2,0,1,5,11,00,20130104,85610,QW,,T1H,5.40,0,5.40,0.00,,0,0.00,8.00,5.40,A,1,1,1,3,42731,9,0,,NA,10D0979077,0
100000099,174,1,20130118,71,49196Z,,1598867251,NA,1,10,11,2,0,1,1,11,03,20130118,99211,,,M1B,0.00,0,0.00,21.46,,0,0.00,37.00,21.46,A,0,0,1,3,42731,9,0,,NA,,0
100000099,174,2,20130118,71,49196Z,,1598867251,NA,1,10,11,2,0,1,5,11,00,20130118,85610,QW,,T1H,5.40,0,5.40,0.00,,0,0.00,8.00,5.40,A,1,1,1,3,42731,9,0,,NA,10D0979077,0
100000099,175,1,20130214,71,49196Z,,1598867251,NA,1,10,11,2,0,1,1,11,03,20130214,99211,,,M1B,0.00,0,0.00,21.46,,0,0.00,37.00,21.46,A,0,0,1,3,42731,9,0,,NA,,0
100000099,175,2,20130214,71,49196Z,,1598867251,NA,1,10,11,2,0,1,5,11,00,20130214,85610,QW,,T1H,5.40,0,5.40,0.00,,0,0.00,8.00,5.40,A,1,1,1,3,42731,9,0,,NA,10D0979077,0
100000099,176,1,20130228,71,49196Z,,1598867251,NA,1,10,11,2,0,1,1,11,03,20130228,99211,,,M1B,0.00,0,0.00,21.46,,0,0.00,37.00,21.46,A,0,0,1,3,42731,9,0,,NA,,0
100000099,176,2,20130228,71,49196Z,,1598867251,NA,1,10,11,2,0,1,5,11,00,20130228,85610,QW,,T1H,5.40,0,5.40,0.00,,0,0.00,8.00,5.40,A,1,1,1,3,42731,9,0,,NA,10D0979077,0
100000099,177,1,20130314,71,49196Z,,1598867251,NA,1,10,11,2,0,1,1,11,03,20130314,99211,,,M1B,0.00,0,0.00,21.46,,0,0.00,37.00,21.46,A,0,0,1,3,42731,9,0,,NA,,0


DESY_SORT_KEY,CLAIM_NO,PRVDR_NUM,CLM_THRU_DT,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLAIM_QUERY_CODE,CLM_FAC_TYPE_CD,CLM_SRVC_CLSFCTN_TYPE_CD,CLM_FREQ_CD,FI_NUM,CLM_MDCR_NON_PMT_RSN_CD,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_PRMRY_PYR_CD,FI_CLM_ACTN_CD,PRVDR_STATE_CD,ORG_NPI_NUM,AT_PHYSN_UPIN,AT_PHYSN_NPI,OP_PHYSN_UPIN,OP_PHYSN_NPI,OT_PHYSN_UPIN,OT_PHYSN_NPI,CLM_MCO_PD_SW,PTNT_DSCHRG_STUS_CD,CLM_PPS_IND_CD,CLM_TOT_CHRG_AMT,CLM_ADMSN_DT,CLM_IP_ADMSN_TYPE_CD,CLM_SRC_IP_ADMSN_CD,NCH_PTNT_STATUS_IND_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_PROFNL_CMPNT_CHRG_AMT,NCH_IP_NCVRD_CHRG_AMT,CLM_TOT_PPS_CPTL_AMT,CLM_PPS_CPTL_FSP_AMT,CLM_PPS_CPTL_OUTLIER_AMT,CLM_PPS_CPTL_DSPRPRTNT_SHR_AMT,CLM_PPS_CPTL_IME_AMT,CLM_PPS_CPTL_EXCPTN_AMT,CLM_PPS_OLD_CPTL_HLD_HRMLS_AMT,CLM_PPS_CPTL_DRG_WT_NUM,CLM_UTLZTN_DAY_CNT,BENE_TOT_COINSRNC_DAYS_CNT,BENE_LRD_USED_CNT,CLM_NON_UTLZTN_DAYS_CNT,NCH_BLOOD_PNTS_FRNSHD_QTY,NCH_VRFD_NCVRD_STAY_FROM_DT,NCH_VRFD_NCVRD_STAY_THRU_DT,NCH_BENE_MDCR_BNFTS_EXHTD_DT_I,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,CLM_DRG_OUTLIER_STAY_CD,NCH_DRG_OUTLIER_APRVD_PMT_AMT,ADMTG_DGNS_CD,ADMTG_DGNS_VRSN_CD,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,CLM_POA_IND_SW1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,CLM_POA_IND_SW2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,CLM_POA_IND_SW3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,CLM_POA_IND_SW4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,CLM_POA_IND_SW5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,CLM_POA_IND_SW6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,CLM_POA_IND_SW7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,CLM_POA_IND_SW8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,CLM_POA_IND_SW9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,CLM_POA_IND_SW10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,CLM_POA_IND_SW11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,CLM_POA_IND_SW12,ICD_DGNS_CD13,ICD_DGNS_VRSN_CD13,CLM_POA_IND_SW13,ICD_DGNS_CD14,ICD_DGNS_VRSN_CD14,CLM_POA_IND_SW14,ICD_DGNS_CD15,ICD_DGNS_VRSN_CD15,CLM_POA_IND_SW15,ICD_DGNS_CD16,ICD_DGNS_VRSN_CD16,CLM_POA_IND_SW16,ICD_DGNS_CD17,ICD_DGNS_VRSN_CD17,CLM_POA_IND_SW17,ICD_DGNS_CD18,ICD_DGNS_VRSN_CD18,CLM_POA_IND_SW18,ICD_DGNS_CD19,ICD_DGNS_VRSN_CD19,CLM_POA_IND_SW19,ICD_DGNS_CD20,ICD_DGNS_VRSN_CD20,CLM_POA_IND_SW20,ICD_DGNS_CD21,ICD_DGNS_VRSN_CD21,CLM_POA_IND_SW21,ICD_DGNS_CD22,ICD_DGNS_VRSN_CD22,CLM_POA_IND_SW22,ICD_DGNS_CD23,ICD_DGNS_VRSN_CD23,CLM_POA_IND_SW23,ICD_DGNS_CD24,ICD_DGNS_VRSN_CD24,CLM_POA_IND_SW24,ICD_DGNS_CD25,ICD_DGNS_VRSN_CD25,CLM_POA_IND_SW25,FST_DGNS_E_CD,FST_DGNS_E_VRSN_CD,ICD_DGNS_E_CD1,ICD_DGNS_E_VRSN_CD1,CLM_E_POA_IND_SW1,ICD_DGNS_E_CD2,ICD_DGNS_E_VRSN_CD2,CLM_E_POA_IND_SW2,ICD_DGNS_E_CD3,ICD_DGNS_E_VRSN_CD3,CLM_E_POA_IND_SW3,ICD_DGNS_E_CD4,ICD_DGNS_E_VRSN_CD4,CLM_E_POA_IND_SW4,ICD_DGNS_E_CD5,ICD_DGNS_E_VRSN_CD5,CLM_E_POA_IND_SW5,ICD_DGNS_E_CD6,ICD_DGNS_E_VRSN_CD6,CLM_E_POA_IND_SW6,ICD_DGNS_E_CD7,ICD_DGNS_E_VRSN_CD7,CLM_E_POA_IND_SW7,ICD_DGNS_E_CD8,ICD_DGNS_E_VRSN_CD8,CLM_E_POA_IND_SW8,ICD_DGNS_E_CD9,ICD_DGNS_E_VRSN_CD9,CLM_E_POA_IND_SW9,ICD_DGNS_E_CD10,ICD_DGNS_E_VRSN_CD10,CLM_E_POA_IND_SW10,ICD_DGNS_E_CD11,ICD_DGNS_E_VRSN_CD11,CLM_E_POA_IND_SW11,ICD_DGNS_E_CD12,ICD_DGNS_E_VRSN_CD12,CLM_E_POA_IND_SW12,ICD_PRCDR_CD1,ICD_PRCDR_VRSN_CD1,PRCDR_DT1,ICD_PRCDR_CD2,ICD_PRCDR_VRSN_CD2,PRCDR_DT2,ICD_PRCDR_CD3,ICD_PRCDR_VRSN_CD3,PRCDR_DT3,ICD_PRCDR_CD4,ICD_PRCDR_VRSN_CD4,PRCDR_DT4,ICD_PRCDR_CD5,ICD_PRCDR_VRSN_CD5,PRCDR_DT5,ICD_PRCDR_CD6,ICD_PRCDR_VRSN_CD6,PRCDR_DT6,ICD_PRCDR_CD7,ICD_PRCDR_VRSN_CD7,PRCDR_DT7,ICD_PRCDR_CD8,ICD_PRCDR_VRSN_CD8,PRCDR_DT8,ICD_PRCDR_CD9,ICD_PRCDR_VRSN_CD9,PRCDR_DT9,ICD_PRCDR_CD10,ICD_PRCDR_VRSN_CD10,PRCDR_DT10,ICD_PRCDR_CD11,ICD_PRCDR_VRSN_CD11,PRCDR_DT11,ICD_PRCDR_CD12,ICD_PRCDR_VRSN_CD12,PRCDR_DT12,ICD_PRCDR_CD13,ICD_PRCDR_VRSN_CD13,PRCDR_DT13,ICD_PRCDR_CD14,ICD_PRCDR_VRSN_CD14,PRCDR_DT14,ICD_PRCDR_CD15,ICD_PRCDR_VRSN_CD15,PRCDR_DT15,ICD_PRCDR_CD16,ICD_PRCDR_VRSN_CD16,PRCDR_DT16,ICD_PRCDR_CD17,ICD_PRCDR_VRSN_CD17,PRCDR_DT17,ICD_PRCDR_CD18,ICD_PRCDR_VRSN_CD18,PRCDR_DT18,ICD_PRCDR_CD19,ICD_PRCDR_VRSN_CD19,PRCDR_DT19,ICD_PRCDR_CD20,ICD_PRCDR_VRSN_

DESY_SORT_KEY,CLAIM_NO,PRVDR_NUM,CLM_THRU_DT,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLAIM_QUERY_CODE,CLM_FAC_TYPE_CD,CLM_SRVC_CLSFCTN_TYPE_CD,CLM_FREQ_CD,FI_NUM,CLM_MDCR_NON_PMT_RSN_CD,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_PRMRY_PYR_CD,PRVDR_STATE_CD,ORG_NPI_NUM,AT_PHYSN_UPIN,AT_PHYSN_NPI,OP_PHYSN_UPIN,OP_PHYSN_NPI,OT_PHYSN_UPIN,OT_PHYSN_NPI,CLM_MCO_PD_SW,PTNT_DSCHRG_STUS_CD,CLM_TOT_CHRG_AMT,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_PROFNL_CMPNT_CHRG_AMT,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,ICD_DGNS_CD13,ICD_DGNS_VRSN_CD13,ICD_DGNS_CD14,ICD_DGNS_VRSN_CD14,ICD_DGNS_CD15,ICD_DGNS_VRSN_CD15,ICD_DGNS_CD16,ICD_DGNS_VRSN_CD16,ICD_DGNS_CD17,ICD_DGNS_VRSN_CD17,ICD_DGNS_CD18,ICD_DGNS_VRSN_CD18,ICD_DGNS_CD19,ICD_DGNS_VRSN_CD19,ICD_DGNS_CD20,ICD_DGNS_VRSN_CD20,ICD_DGNS_CD21,ICD_DGNS_VRSN_CD21,ICD_DGNS_CD22,ICD_DGNS_VRSN_CD22,ICD_DGNS_CD23,ICD_DGNS_VRSN_CD23,ICD_DGNS_CD24,ICD_DGNS_VRSN_CD24,ICD_DGNS_CD25,ICD_DGNS_VRSN_CD25,FST_DGNS_E_CD,FST_DGNS_E_VRSN_CD,ICD_DGNS_E_CD1,ICD_DGNS_E_VRSN_CD1,ICD_DGNS_E_CD2,ICD_DGNS_E_VRSN_CD2,ICD_DGNS_E_CD3,ICD_DGNS_E_VRSN_CD3,ICD_DGNS_E_CD4,ICD_DGNS_E_VRSN_CD4,ICD_DGNS_E_CD5,ICD_DGNS_E_VRSN_CD5,ICD_DGNS_E_CD6,ICD_DGNS_E_VRSN_CD6,ICD_DGNS_E_CD7,ICD_DGNS_E_VRSN_CD7,ICD_DGNS_E_CD8,ICD_DGNS_E_VRSN_CD8,ICD_DGNS_E_CD9,ICD_DGNS_E_VRSN_CD9,ICD_DGNS_E_CD10,ICD_DGNS_E_VRSN_CD10,ICD_DGNS_E_CD11,ICD_DGNS_E_VRSN_CD11,ICD_DGNS_E_CD12,ICD_DGNS_E_VRSN_CD12,ICD_PRCDR_CD1,ICD_PRCDR_VRSN_CD1,PRCDR_DT1,ICD_PRCDR_CD2,ICD_PRCDR_VRSN_CD2,PRCDR_DT2,ICD_PRCDR_CD3,ICD_PRCDR_VRSN_CD3,PRCDR_DT3,ICD_PRCDR_CD4,ICD_PRCDR_VRSN_CD4,PRCDR_DT4,ICD_PRCDR_CD5,ICD_PRCDR_VRSN_CD5,PRCDR_DT5,ICD_PRCDR_CD6,ICD_PRCDR_VRSN_CD6,PRCDR_DT6,ICD_PRCDR_CD7,ICD_PRCDR_VRSN_CD7,PRCDR_DT7,ICD_PRCDR_CD8,ICD_PRCDR_VRSN_CD8,PRCDR_DT8,ICD_PRCDR_CD9,ICD_PRCDR_VRSN_CD9,PRCDR_DT9,ICD_PRCDR_CD10,ICD_PRCDR_VRSN_CD10,PRCDR_DT10,ICD_PRCDR_CD11,ICD_PRCDR_VRSN_CD11,PRCDR_DT11,ICD_PRCDR_CD12,ICD_PRCDR_VRSN_CD12,PRCDR_DT12,ICD_PRCDR_CD13,ICD_PRCDR_VRSN_CD13,PRCDR_DT13,ICD_PRCDR_CD14,ICD_PRCDR_VRSN_CD14,PRCDR_DT14,ICD_PRCDR_CD15,ICD_PRCDR_VRSN_CD15,PRCDR_DT15,ICD_PRCDR_CD16,ICD_PRCDR_VRSN_CD16,PRCDR_DT16,ICD_PRCDR_CD17,ICD_PRCDR_VRSN_CD17,PRCDR_DT17,ICD_PRCDR_CD18,ICD_PRCDR_VRSN_CD18,PRCDR_DT18,ICD_PRCDR_CD19,ICD_PRCDR_VRSN_CD19,PRCDR_DT19,ICD_PRCDR_CD20,ICD_PRCDR_VRSN_CD20,PRCDR_DT20,ICD_PRCDR_CD21,ICD_PRCDR_VRSN_CD21,PRCDR_DT21,ICD_PRCDR_CD22,ICD_PRCDR_VRSN_CD22,PRCDR_DT22,ICD_PRCDR_CD23,ICD_PRCDR_VRSN_CD23,PRCDR_DT23,ICD_PRCDR_CD24,ICD_PRCDR_VRSN_CD24,PRCDR_DT24,ICD_PRCDR_CD25,ICD_PRCDR_VRSN_CD25,PRCDR_DT25,RSN_VISIT_CD1,RSN_VISIT_VRSN_CD1,RSN_VISIT_CD2,RSN_VISIT_VRSN_CD2,RSN_VISIT_CD3,RSN_VISIT_VRSN_CD3,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,CLM_OP_PRVDR_PMT_AMT,CLM_OP_BENE_PMT_AMT,DOB_DT,GNDR_CD,BENE_RACE_CD,BENE_CNTY_CD,BENE_STATE_CD,CWF_BENE_MDCR_STUS_CD,FI_CLM_ACTN_CD
<int>,<int>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<lgl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>

## Reading MBSF and revenue center files

### MBSF

In [2]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table


New names:
* `1` -> `1...5`
* `1` -> `1...7`
* `0` -> `0...8`
* `0` -> `0...9`
* `0` -> `0...11`
* ...

Rows: 2758309 Columns: 43

── Column specification ───────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17,...
dbl (14): 2, 1...5, 70, 1...7, 0...8, 10, 0...11, 0...12, 12...37, 12...38, ...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `1` -> `1...5`
* `1` -> `1...7`
* `0` -> `0...8`
* `0` -> `0...9`
* `0` -> `0...11`
* ...

Rows: 2836378 Columns: 43

── Column specification ───────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17,...
dbl (14): 2, 1...5, 71, 1...7, 0...8, 10, 0...11, 0...12, 12...37, 12...38, ...


ℹ Use `spec()

In [9]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [25]:
mbsf_2013[DESY_SORT_KEY==100000905]

DESY_SORT_KEY,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ESRD_INDICATOR,MEDICARE_STATUS_CD,PART_A_TERMINATION_CODE,PART_B_TERMINATION_CODE,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,HI_COVERAGE,SMI_COVERAGE,HMO_COVERAGE,STATE_BUY_IN_COVERAGE,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,REFERENCE_YEAR
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
100000905,22,040,2,1,76,0,0,0,10,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0,0,NA,NA,13


### Revenue center files

In [103]:
revenue_center_2013_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2014_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2015_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuej_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2016_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2017_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2018_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2019_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table()
revenue_center_2020_outpatient=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/Outpatient/out_revenuek_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table()



New names:
* `1` -> `1...2`
* `1` -> `1...3`
* `20131105` -> `20131105...4`
* `20131105` -> `20131105...7`
* `` -> ...10
* ...

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 56454360 Columns: 32

── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): 0300, 00000, 36415, ...10, ...11, ...16, A, B
dbl (23): 100000015, 1...2, 1...3, 20131105...4, 40, 20131105...7, 2, 1...13...
lgl  (1): ...15


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

New names:
* `1` -> `1...2`
* `1` -> `1...3`
* `20140123` -> `20140123...4`
* `20140123` -> `20140123...7`
* `` -> ...10
* ...

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 58178971 Columns: 

In [104]:
colnames_revenue_center_outpatient_2013_2015=c("DESY_SORT_KEY","CLAIM_NO","CLM_LINE_NUM","CLM_THRU_DT","NCH_CLM_TYPE_CD","REV_CNTR","REV_CNTR_DT","REV_CNTR_APC_HIPPS_CD","HCPCS_CD","HCPCS_1ST_MDFR_CD","HCPCS_2ND_MDFR_CD","REV_CNTR_PMT_MTHD_IND_CD","REV_CNTR_DSCNT_IND_CD","REV_CNTR_PACKG_IND_CD","REV_CNTR_OTAF_PMT_CD","REV_CNTR_IDE_NDC_UPC_NUM","REV_CNTR_UNIT_CNT","REV_CNTR_RATE_AMT","REV_CNTR_BLOOD_DDCTBL_AMT","REV_CNTR_CASH_DDCTBL_AMT","REV_CNTR_COINSRNC_WGE_ADJSTD_C","REV_CNTR_RDCD_COINSRNC_AMT","REV_CNTR_1ST_MSP_PD_AMT","REV_CNTR_2ND_MSP_PD_AMT","REV_CNTR_PRVDR_PMT_AMT","REV_CNTR_BENE_PMT_AMT","REV_CNTR_PTNT_RSPNSBLTY_PMT","REV_CNTR_PMT_AMT_AMT","REV_CNTR_TOT_CHRG_AMT","REV_CNTR_NCVRD_CHRG_AMT","REV_CNTR_STUS_IND_CD","REV_CNTR_PRICNG_IND_CD")

colnames_revenue_center_outpatient_2016_2020=c("DESY_SORT_KEY","CLAIM_NO","CLM_LINE_NUM","CLM_THRU_DT","NCH_CLM_TYPE_CD","REV_CNTR","REV_CNTR_DT","REV_CNTR_APC_HIPPS_CD","HCPCS_CD","HCPCS_1ST_MDFR_CD","HCPCS_2ND_MDFR_CD","HCPCS_3RD_MDFR_CD","HCPCS_4TH_MDFR_CD","REV_CNTR_PMT_MTHD_IND_CD","REV_CNTR_DSCNT_IND_CD","REV_CNTR_PACKG_IND_CD","REV_CNTR_OTAF_PMT_CD","REV_CNTR_IDE_NDC_UPC_NUM","REV_CNTR_UNIT_CNT","REV_CNTR_RATE_AMT","REV_CNTR_BLOOD_DDCTBL_AMT","REV_CNTR_CASH_DDCTBL_AMT","REV_CNTR_COINSRNC_WGE_ADJSTD_C","REV_CNTR_RDCD_COINSRNC_AMT","REV_CNTR_1ST_MSP_PD_AMT","REV_CNTR_2ND_MSP_PD_AMT","REV_CNTR_PRVDR_PMT_AMT","REV_CNTR_BENE_PMT_AMT","REV_CNTR_PTNT_RSPNSBLTY_PMT","REV_CNTR_PMT_AMT_AMT","REV_CNTR_TOT_CHRG_AMT","REV_CNTR_NCVRD_CHRG_AMT","REV_CNTR_STUS_IND_CD","REV_CNTR_PRICNG_IND_CD","REV_CNTR_RNDRNG_PHYSN_UPIN","REV_CNTR_RNDRNG_PHYSN_NPI","REV_CNTR_RNDRNG_PHYSN_SPCLTY_CD","REV_CNTR_DDCTBL_COINSRNC_CD","THRPY_CAP_IND_CD1","THRPY_CAP_IND_CD2","RC_PTNT_ADD_ON_PYMT_AMT")

colnames(revenue_center_2013_outpatient)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_2014_outpatient)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_2015_outpatient)=colnames_revenue_center_outpatient_2013_2015
colnames(revenue_center_2016_outpatient)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_2017_outpatient)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_2018_outpatient)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_2019_outpatient)=colnames_revenue_center_outpatient_2016_2020
colnames(revenue_center_2020_outpatient)=colnames_revenue_center_outpatient_2016_2020



In [105]:
head(revenue_center_2013_outpatient)

DESY_SORT_KEY,CLAIM_NO,CLM_LINE_NUM,CLM_THRU_DT,NCH_CLM_TYPE_CD,REV_CNTR,REV_CNTR_DT,REV_CNTR_APC_HIPPS_CD,HCPCS_CD,HCPCS_1ST_MDFR_CD,HCPCS_2ND_MDFR_CD,REV_CNTR_PMT_MTHD_IND_CD,REV_CNTR_DSCNT_IND_CD,REV_CNTR_PACKG_IND_CD,REV_CNTR_OTAF_PMT_CD,REV_CNTR_IDE_NDC_UPC_NUM,REV_CNTR_UNIT_CNT,REV_CNTR_RATE_AMT,REV_CNTR_BLOOD_DDCTBL_AMT,REV_CNTR_CASH_DDCTBL_AMT,REV_CNTR_COINSRNC_WGE_ADJSTD_C,REV_CNTR_RDCD_COINSRNC_AMT,REV_CNTR_1ST_MSP_PD_AMT,REV_CNTR_2ND_MSP_PD_AMT,REV_CNTR_PRVDR_PMT_AMT,REV_CNTR_BENE_PMT_AMT,REV_CNTR_PTNT_RSPNSBLTY_PMT,REV_CNTR_PMT_AMT_AMT,REV_CNTR_TOT_CHRG_AMT,REV_CNTR_NCVRD_CHRG_AMT,REV_CNTR_STUS_IND_CD,REV_CNTR_PRICNG_IND_CD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1e+08,1,2,20131105,40,0301,20131105,00000,80048,NA,NA,2,1,0,NA,NA,1,11.63,0,0,0.00,0,0,0,7.15,0,0.00,7.15,53.9,0,A,B
1e+08,1,3,20131105,40,0301,20131105,00000,80061,NA,NA,2,1,0,NA,NA,1,18.42,0,0,0.00,0,0,0,11.32,0,0.00,11.32,63.0,0,A,B
1e+08,1,4,20131105,40,0301,20131105,00000,80076,NA,NA,2,1,0,NA,NA,1,11.23,0,0,0.00,0,0,0,6.90,0,0.00,6.90,63.8,0,A,B
1e+08,1,5,20131105,40,0301,20131105,00000,83036,NA,NA,2,1,0,NA,NA,1,13.34,0,0,0.00,0,0,0,13.07,0,0.00,13.07,40.0,0,A,B
1e+08,1,6,20131105,40,0324,20131105,00260,71020,NA,NA,1,1,0,NA,NA,1,0.00,0,0,11.09,0,0,0,43.45,0,11.09,43.45,133.0,0,X,NA
1e+08,1,7,20131105,40,0001,NA,00000,NA,NA,NA,2,NA,NA,NA,NA,0,0.00,0,0,0.00,0,0,0,0.00,0,0.00,0.00,376.4,0,A,NA


## Choosing columns and combine years

A function to choose the needed columns from our raw Medicare files and also combining all years in one file.

### Carrier, inpatient, and outpatient

In [64]:
choose_columns = function(data_list,
                          columns = c(
                            "DESY_SORT_KEY",
                            "CLAIM_NO",
                            "LINE_NUM",
                            "CLM_THRU_DT",
                            "LINE_PLACE_OF_SRVC_CD",
                            "HCPCS_CD",
                            "LINE_ICD_DGNS_VRSN_CD",
                            "LINE_ICD_DGNS_CD",
                            "LINE_ALOWD_CHRG_AMT",
                            "PRF_PHYSN_NPI",
                            "PRVDR_SPCLTY",
                            "PRVDR_STATE_CD"
                          )) {
  require(data.table)
  data_list = lapply(data_list, function (data)
    data [, ..columns])
  result = rbindlist(data_list)
  return(result)
}

#carrier_data_all_years = choose_columns(
#  list(
#    carrier_data_2013,
#    carrier_data_2014,
#    carrier_data_2015,
#    carrier_data_2016,
#    carrier_data_2017,
#    carrier_data_2018,
#    carrier_data_2019,
#    carrier_data_2020
#  )
#)
#
#carrier_data_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
#carrier_data_all_years[,month_year := format(date, "%Y-%m")]
#                     
#write_fst(carrier_data_all_years, "carrier_data_all_years.fst")

#outpatient_data_2013_2015 = choose_columns(
#  data_list = list(
#    outpatient_data_2013,
#    outpatient_data_2014,
#    outpatient_data_2015
#  ),
#  columns = c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "CLM_TOT_CHRG_AMT","PRNCPAL_DGNS_CD","PRNCPAL_DGNS_VRSN_CD")
#)
#outpatient_data_2013_2015[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
#
#outpatient_data_2016_2020 = choose_columns(
#  data_list = list(
#    outpatient_data_2016,
#    outpatient_data_2017,
#    outpatient_data_2018,
#    outpatient_data_2019,
#    outpatient_data_2020
#  ),
#  columns = c("DESY_SORT_KEY", "CLM_THRU_DT", "CLM_TOT_CHRG_AMT","PRNCPAL_DGNS_CD")
#)
#outpatient_data_2016_2020[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
#
#outpatient_data_all_years=rbind(outpatient_data_2013_2015,
#                                outpatient_data_2016_2020,fill=T)%>%
#                           mutate(PRNCPAL_DGNS_VRSN_CD=replace_na(PRNCPAL_DGNS_VRSN_CD,as.integer(0)))%>%as.data.table
#                     
#outpatient_data_all_years                     
#write_fst(outpatient_data_all_years, "outpatient_data_all_years.fst")

inpatient_data_2013_2015 = choose_columns(
  data_list = list(
    inpatient_data_2013,
    inpatient_data_2014,
    inpatient_data_2015
  ),
  columns = c("DESY_SORT_KEY", "CLM_THRU_DT", "CLM_TOT_CHRG_AMT","PRNCPAL_DGNS_CD","PRNCPAL_DGNS_VRSN_CD","CLM_DRG_CD")
)
inpatient_data_2013_2015[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]

inpatient_data_2016_2020 = choose_columns(
  data_list = list(
    inpatient_data_2016,
    inpatient_data_2017,
    inpatient_data_2018,
    inpatient_data_2019,
    inpatient_data_2020
  ),
  columns = c("DESY_SORT_KEY", "CLM_THRU_DT", "CLM_TOT_CHRG_AMT","PRNCPAL_DGNS_CD","CLM_DRG_CD")
)
inpatient_data_2016_2020[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]

inpatient_data_all_years=rbind(inpatient_data_2013_2015,
                                inpatient_data_2016_2020,fill=T)%>%
                           mutate(PRNCPAL_DGNS_VRSN_CD=replace_na(PRNCPAL_DGNS_VRSN_CD,as.integer(0)))%>%as.data.table 
                
                     
write_fst(inpatient_data_all_years, "inpatient_data_all_years.fst")
                     

DESY_SORT_KEY,CLM_THRU_DT,CLM_TOT_CHRG_AMT,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,CLM_DRG_CD,date
<int>,<int>,<dbl>,<chr>,<int>,<int>,<date>
100153533,20130101,19349.20,0389,9,871,2013-01-01
100222555,20130101,2766.70,78650,9,313,2013-01-01
100335009,20130101,40864.68,51881,9,208,2013-01-01
100393077,20130101,15555.00,29421,9,884,2013-01-01
100397165,20130101,20829.20,5990,9,690,2013-01-01
100408803,20130101,25413.25,7384,9,552,2013-01-01
100425189,20130101,76868.00,57461,9,444,2013-01-01
100540009,20130101,9626.43,5849,9,683,2013-01-01
100554011,20130101,27640.40,486,9,194,2013-01-01


DESY_SORT_KEY,CLM_THRU_DT,CLM_TOT_CHRG_AMT,PRNCPAL_DGNS_CD,CLM_DRG_CD,date
<int>,<int>,<dbl>,<chr>,<int>,<date>
100093217,20160101,107600.00,S72451A,481,2016-01-01
100097835,20160101,14677.06,N178,682,2016-01-01
100121297,20160101,99816.50,A419,854,2016-01-01
100274463,20160101,236784.45,A4151,870,2016-01-01
100367273,20160101,111950.25,A021,854,2016-01-01
100396827,20160101,8341.74,T8454XA,560,2016-01-01
100454975,20160101,14991.15,I214,282,2016-01-01
100596793,20160101,75014.41,C186,329,2016-01-01
100620981,20160101,8610.72,J209,202,2016-01-01


### Revenue center

In [107]:
choose_columns = function(data_list,columns) {
  require(data.table)
  data_list = lapply(data_list, function (data) { data [, ..columns] } )
  result = rbindlist(data_list)
  return(result)
}

revenue_center_outpatient_all_years = choose_columns(
  list(
  revenue_center_2013_outpatient,
  revenue_center_2014_outpatient,
  revenue_center_2015_outpatient,
  revenue_center_2016_outpatient,
  revenue_center_2017_outpatient,
  revenue_center_2018_outpatient,
  revenue_center_2019_outpatient,
  revenue_center_2020_outpatient),
  columns=c("DESY_SORT_KEY","CLAIM_NO","CLM_THRU_DT","REV_CNTR","HCPCS_CD","REV_CNTR_TOT_CHRG_AMT"))


revenue_center_outpatient_all_years[,date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
revenue_center_outpatient_all_years[,month_year := format(date, "%Y-%m")]
revenue_center_outpatient_all_years[,DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]

DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,REV_CNTR,HCPCS_CD,REV_CNTR_TOT_CHRG_AMT,date
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<date>
100019625,2332,20130101,0300,36415,6.50,2013-01-01
100019625,2332,20130101,0301,85025,47.86,2013-01-01
100019625,2332,20130101,0001,NA,54.36,2013-01-01
100020607,2405,20130101,0260,96365,649.00,2013-01-01
100020607,2405,20130101,0636,J1335,459.10,2013-01-01
100020607,2405,20130101,0001,NA,1108.10,2013-01-01
100023251,2604,20130101,0540,A0430,6473.85,2013-01-01
100023251,2604,20130101,0540,A0435,5568.30,2013-01-01
100023251,2604,20130101,0001,NA,12042.15,2013-01-01


In [ ]:
head(revenue_center_outpatient_all_years)

DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,REV_CNTR,HCPCS_CD,REV_CNTR_TOT_CHRG_AMT,date,month_year
<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<date>,<chr>
100000015,1,20131105,0301,80048,53.9,2013-11-05,2013-11
100000015,1,20131105,0301,80061,63.0,2013-11-05,2013-11
100000015,1,20131105,0301,80076,63.8,2013-11-05,2013-11
100000015,1,20131105,0301,83036,40.0,2013-11-05,2013-11
100000015,1,20131105,0324,71020,133.0,2013-11-05,2013-11
100000015,1,20131105,0001,NA,376.4,2013-11-05,2013-11


In [ ]:
write_fst(revenue_center_outpatient_all_years, "/work/postresearch/Shared/Projects/Data_fst/revenue_center_outpatient_all_years.fst")

### Joining the outpatient and outpatient revenue files together

In [117]:
revenue_center_outpatient_all_years[,year:=year(date)]
outpatient_data_all_years[,year:=year(date)]
outpatient_and_revenue_center_data=left_join(
  revenue_center_outpatient_all_years,
  outpatient_data_all_years[,-c("CLM_THRU_DT","date")],
  by=c("DESY_SORT_KEY","CLAIM_NO","year"))%>%
  as.data.table()

head(outpatient_and_revenue_center_data)

DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,REV_CNTR,HCPCS_CD,REV_CNTR_TOT_CHRG_AMT,date,month_year,year,CLM_TOT_CHRG_AMT,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD
<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<date>,<chr>,<dbl>,<dbl>,<chr>,<int>
100000015,1,20131105,0301,80048,53.9,2013-11-05,2013-11,2013,NA,NA,NA
100000015,1,20131105,0301,80061,63.0,2013-11-05,2013-11,2013,NA,NA,NA
100000015,1,20131105,0301,80076,63.8,2013-11-05,2013-11,2013,NA,NA,NA
100000015,1,20131105,0301,83036,40.0,2013-11-05,2013-11,2013,NA,NA,NA
100000015,1,20131105,0324,71020,133.0,2013-11-05,2013-11,2013,NA,NA,NA
100000015,1,20131105,0001,NA,376.4,2013-11-05,2013-11,2013,NA,NA,NA


In [119]:
write_fst(outpatient_and_revenue_center_data, "/work/postresearch/Shared/Projects/Data_fst/outpatient_and_revenue_center_data.fst")

### MBSF
Here, we need to follow each patient across the years in the data set. So, our aproach will differ.

In [31]:
mbsf_needed_cols = c(
    "DESY_SORT_KEY"
    ,"REFERENCE_YEAR"
    ,"STATE_CODE"
    ,"COUNTY_CODE"
    ,"SEX_CODE"
    ,"RACE_CODE"
    ,"AGE"
    ,"ORIG_REASON_FOR_ENTITLEMENT"
    ,"CURR_REASON_FOR_ENTITLEMENT"
    ,"ENTITLEMENT_BUY_IN_IND01"
    ,"ENTITLEMENT_BUY_IN_IND02"
    ,"ENTITLEMENT_BUY_IN_IND03"
    ,"ENTITLEMENT_BUY_IN_IND04"
    ,"ENTITLEMENT_BUY_IN_IND05"
    ,"ENTITLEMENT_BUY_IN_IND06"
    ,"ENTITLEMENT_BUY_IN_IND07"
    ,"ENTITLEMENT_BUY_IN_IND08"
    ,"ENTITLEMENT_BUY_IN_IND09"
    ,"ENTITLEMENT_BUY_IN_IND10"
    ,"ENTITLEMENT_BUY_IN_IND11"
    ,"ENTITLEMENT_BUY_IN_IND12"
    ,"HMO_INDICATOR01"
    ,"HMO_INDICATOR02"
    ,"HMO_INDICATOR03"
    ,"HMO_INDICATOR04"
    ,"HMO_INDICATOR05"
    ,"HMO_INDICATOR06"
    ,"HMO_INDICATOR07"
    ,"HMO_INDICATOR08"
    ,"HMO_INDICATOR09"
    ,"HMO_INDICATOR10"
    ,"HMO_INDICATOR11"
    ,"HMO_INDICATOR12"
    ,"VALID_DATE_OF_DEATH_SWITCH"
    ,"DATE_OF_DEATH")

mbsf_data=list(mbsf_2013,mbsf_2014,mbsf_2015,mbsf_2016,mbsf_2017,mbsf_2018,mbsf_2019,mbsf_2020)

for (a in 1:length(mbsf_data)){
  mbsf_data[[a]]=mbsf_data[[a]][, ..mbsf_needed_cols]
}

#rename last n columns in a dataset
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}

for (a in 1:length(mbsf_data)){
  rename_last(
    mbsf_data[[a]],
    ncol(mbsf_data[[a]])-1,
    paste(colnames(mbsf_data[[a]])[2:ncol(mbsf_data[[a]])],paste("_",as.character(2012+a),sep=""),sep=""))
}

for (a in 1:length(mbsf_data)){
  mbsf_data[[a]][,DESY_SORT_KEY:=as.integer(DESY_SORT_KEY)]
}



mbsf_data = reduce(mbsf_data, function(x,y) {full_join(x,y,by="DESY_SORT_KEY")}) %>% as.data.table()
                   


In [33]:
tail(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,⋯,ENTITLEMENT_BUY_IN_IND11_2019,ENTITLEMENT_BUY_IN_IND12_2019,HMO_INDICATOR01_2019,HMO_INDICATOR02_2019,HMO_INDICATOR03_2019,HMO_INDICATOR04_2019,HMO_INDICATOR05_2019,HMO_INDICATOR06_2019,HMO_INDICATOR07_2019,HMO_INDICATOR08_2019,HMO_INDICATOR09_2019,HMO_INDICATOR10_2019,HMO_INDICATOR11_2019,HMO_INDICATOR12_2019,VALID_DATE_OF_DEATH_SWITCH_2019,DATE_OF_DEATH_2019,REFERENCE_YEAR_2020,STATE_CODE_2020,COUNTY_CODE_2020,SEX_CODE_2020,RACE_CODE_2020,AGE_2020,ORIG_REASON_FOR_ENTITLEMENT_2020,CURR_REASON_FOR_ENTITLEMENT_2020,ENTITLEMENT_BUY_IN_IND01_2020,ENTITLEMENT_BUY_IN_IND02_2020,ENTITLEMENT_BUY_IN_IND03_2020,ENTITLEMENT_BUY_IN_IND04_2020,ENTITLEMENT_BUY_IN_IND05_2020,ENTITLEMENT_BUY_IN_IND06_2020,ENTITLEMENT_BUY_IN_IND07_2020,ENTITLEMENT_BUY_IN_IND08_2020,ENTITLEMENT_BUY_IN_IND09_2020,ENTITLEMENT_BUY_IN_IND10_2020,ENTITLEMENT_BUY_IN_IND11_2020,ENTITLEMENT_BUY_IN_IND12_2020,HMO_INDICATOR01_2020,HMO_INDICATOR02_2020,HMO_INDICATOR03_2020,HMO_INDICATOR04_2020,HMO_INDICATOR05_2020,HMO_INDICATOR06_2020,HMO_INDICATOR07_2020,HMO_INDICATOR08_2020,HMO_INDICATOR09_2020,HMO_INDICATOR10_2020,HMO_INDICATOR11_2020,HMO_INDICATOR12_2020,VALID_DATE_OF_DEATH_SWITCH_2020,DATE_OF_DEATH_2020
<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
499999741,13,19,510,1,1,52,1,1,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,14,19,510,1,1,53,1,1,C,C,C,C,C,C,C,⋯,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,2020,19,510,1,1,59,1,1,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999841,13,45,610,1,1,73,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,14,45,610,1,1,74,0,0,3,3,3,3,3,3,3,⋯,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2020,45,610,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999951,13,50,120,2,1,53,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,14,50,120,2,1,54,1,1,C,C,C,C,C,C,C,⋯,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2020,50,120,2,1,60,1,1,C,C,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA
499999961,13,38,250,1,2,75,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,14,38,250,1,2,76,0,0,3,3,3,3,3,3,3,⋯,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA,2020,38,250,1,2,82,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,NA,NA
499999971,13,38,190,2,1,73,0,0,3,3,3,3,3,3,3,3,3,3,3,3,C,C,C,C,C,C,C,C,C,C,C,C,N

Since date of death happens only once for each patient, I will collapse the date of death and its validity to a single column.

In [176]:
mbsf_data_death_collapsed = apply(mbsf_data[,
                                            .(DATE_OF_DEATH_2013,
                                              DATE_OF_DEATH_2014,
                                              DATE_OF_DEATH_2015,
                                              DATE_OF_DEATH_2016,
                                              DATE_OF_DEATH_2017,
                                              DATE_OF_DEATH_2018,
                                              DATE_OF_DEATH_2019,
                                              DATE_OF_DEATH_2020
                                             )
                                           ], 1, function(x){ paste(x[!is.na(x)], collapse = ", ")})

mbsf_data_death_validity_collapsed = apply(mbsf_data[,
                                            .(VALID_DATE_OF_DEATH_SWITCH_2013,
                                              VALID_DATE_OF_DEATH_SWITCH_2014,
                                              VALID_DATE_OF_DEATH_SWITCH_2015,
                                              VALID_DATE_OF_DEATH_SWITCH_2016,
                                              VALID_DATE_OF_DEATH_SWITCH_2017,
                                              VALID_DATE_OF_DEATH_SWITCH_2018,
                                              VALID_DATE_OF_DEATH_SWITCH_2019,
                                              VALID_DATE_OF_DEATH_SWITCH_2020
                                             )
                                           ], 1, function(x){ paste(x[!is.na(x)], collapse = ", ")})


mbsf_data_death_collapsed[mbsf_data_death_collapsed==""]=NA
mbsf_data_death_collapsed=as.IDate(mbsf_data_death_collapsed, "%Y%m%d")

mbsf_data_death_validity_collapsed[mbsf_data_death_validity_collapsed==""]=NA

mbsf_data[, `:=`(date_of_death_collapsed = mbsf_data_death_collapsed,
              death_validity_collapsed = mbsf_data_death_validity_collapsed)]

mbsf_data[!is.na(death_validity_collapsed),death_validity_collapsed:=TRUE]
mbsf_data[is.na(death_validity_collapsed),death_validity_collapsed:=FALSE]

Warning message in `[.data.table`(mbsf_data, , `:=`(date_of_death_collapsed = mbsf_data_death_collapsed, :
“Invalid .internal.selfref detected and fixed by taking a (shallow) copy of the data.table so that := can add this new column by reference. At an earlier point, this data.table has been copied by R (or was created manually using structure() or similar). Avoid names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. If this message doesn't help, please report your use case to the data.table issue tracker so the root cause can be fixed or this message improved.”


In [177]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Data_fst/mbsf_data.fst")

# Patient level calculations

## Yearly Calculators

These are the main functions that calculate yearly expenditures for patients and their corresponding physicians.\

### Read data from choose columns



In [32]:
carrier_data_all_years = read_fst(
    "carrier_data_all_years.fst", as.data.table = T,to = 10000000)

In [4]:
outpatient_data_all_years = read_fst(
    "outpatient_data_all_years.fst", as.data.table = T,to = 1000000)
inpatient_data_all_years = read_fst(
    "inpatient_data_all_years.fst", as.data.table = T,to = 1000000)

In [5]:
mbsf_data = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/mbsf_data", as.data.table = T,to = 1000000)
revenue_center_outpatient_all_years = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/revenue_center_outpatient_all_years.fst", as.data.table = T,to = 1000000)
outpatient_and_revenue_center_data = read_fst(
  "/work/postresearch/Shared/Projects/Data_fst/outpatient_and_revenue_center_data.fst", as.data.table = T,to = 1000000)

In [9]:
#carrier_sample = tail(carrier_data_all_years,1000000)
#outpatient_sample = tail(outpatient_data_all_years,1000000)
#inpatient_sample = tail(inpatient_data_all_years,1000000)
#mbsf_sample = tail(mbsf_data,1000000)
#revenue_center_outpatient_sample=tail(revenue_center_outpatient_all_years,1000000)
#outpatient_and_revenue_center_data_sample=tail(outpatient_and_revenue_center_data,1000000)
#head(carrier_sample)
#head(outpatient_sample)
#head(inpatient_sample)
#head(mbsf_sample)
#head(revenue_center_outpatient_sample)
#head(outpatient_and_revenue_center_data_sample)

### Loading sample data (for pc)

In [1]:
sample_data=readRDS(file = "sample_data.RDS")

In [2]:
carrier_data_all_years=sample_data[[1]]
outpatient_data_all_years=sample_data[[2]]
inpatient_data_all_years=sample_data[[3]]
mbsf_data=sample_data[[4]]
revenue_center_outpatient_all_years=sample_data[[5]]
outpatient_and_revenue_center_data=sample_data[[6]]

In [8]:
carrier_data_all_years=carrier_data_all_years%>%as.data.table()

In [1]:
head(carrier_data_all_years[substr(LINE_ICD_DGNS_CD, 0, 3) == "V85"])

ERROR: Error in head(carrier_data_all_years[substr(LINE_ICD_DGNS_CD, 0, 3) == : object 'carrier_data_all_years' not found


### Patient yearly expenditures and use of services carrier

I will first create a function that adds conditions of interest to the data.


#### Finding conditions for each claim line

In [ ]:
yearly_calculator_patient_conditions = function(data) {
  
  #requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)
  
  data %>%
    mutate(
      is_cardiology_related = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0, 
        substr(LINE_ICD_DGNS_CD, 0, 1) == "I",
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          as.numeric(substr(LINE_ICD_DGNS_CD, 0, 3))>=399 & 
          as.numeric(substr(LINE_ICD_DGNS_CD, 0, 3))<=459 ,NA)),      
      is_catheterization = HCPCS_CD %in% angio_codes,
      is_ecg = HCPCS_CD %in% ecg_codes,
      is_cardiac_ct = HCPCS_CD %in% cardiac_ct_codes,
      is_cardiac_mri = HCPCS_CD %in% cardiac_mri_codes,
      is_stress_test = HCPCS_CD %in% stress_test_codes,
      is_echocardiography = HCPCS_CD %in% echocardiography_codes,
      is_angioplasty = HCPCS_CD %in% angioplasty_codes,
      is_CABG = HCPCS_CD %in% CABG_codes,
      is_stable_angina = ifelse(
        LINE_ICD_DGNS_VRSN_CD == 0,
        LINE_ICD_DGNS_CD %in% c ("I208", "I209"),
        ifelse(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4139", NA)),
      is_unstable_angina = ifelse(
        LINE_ICD_DGNS_VRSN_CD == 0,
        LINE_ICD_DGNS_CD == "I200",
        ifelse(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4111", NA)),
      is_MI = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I21",
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) == "410" ,NA)),
      is_cardiac_arrest = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I46",
        if_else(LINE_ICD_DGNS_VRSN_CD == 9, LINE_ICD_DGNS_CD == "4275", NA)),
      is_stroke = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) == "I63",
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          LINE_ICD_DGNS_CD %in% stroke_icd_9_codes,NA)),
      is_office_visit = HCPCS_CD %in% office_visit_codes,
      is_cardiology_office_vist =
        (HCPCS_CD %in% office_visit_codes) &
        (PRVDR_SPCLTY %in% c("06", "C3")),
      is_IHD = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_9_codes,NA)),
      is_by_cardiologist= PRVDR_SPCLTY %in% c("06","C3"),
      is_by_primary_care_physician= PRVDR_SPCLTY %in% primary_care_specialty_codes,
      is_by_surgeon= PRVDR_SPCLTY %in% surgery_specialty_codes,
      is_hypertension= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes,NA)),
      is_copd= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% copd_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% copd_icd_9_codes,NA)),
      is_obesity= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 4) %in% obesity_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 4) %in% obesity_icd_9_codes,NA)),
      is_depression= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 4) %in% depression_icd_9_codes,NA)),
      is_diabetes= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes,NA))
    ) %>%
    as.data.table()
}

yearly_patient_conditions_carrier=yearly_calculator_patient_conditions(carrier_data_all_years)
head(yearly_patient_conditions_carrier)


Loading required package: data.table

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘data.table’”
Loading required package: dtplyr

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘dtplyr’”
Loading required package: tidyverse

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tidyverse’”
Loading required package: lubridate

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lubridate’”


ERROR: Error in data %>% mutate(is_cardiology_related = if_else(LINE_ICD_DGNS_VRSN_CD == : could not find function "%>%"


#### Summarizing patient data
I will now summarise the data for each patient.


In [42]:
summarise_expenditures_carrier = function(data, time_frame = 365, diagnosis){
  
  data%>%
    group_by(DESY_SORT_KEY) %>%
    filter(sum(eval(parse(
      text = paste("is_", diagnosis, sep = "")
    )), na.rm = T) == T) %>%
    mutate(first_diagnosis = min(date[eval(parse(text = paste("is_", diagnosis, sep = ""))) ==
                                        T]), na.rm = T) %>%
    mutate( had_IHD = (
      date - first_diagnosis < 0 &
        first_diagnosis - date < time_frame &
        is_IHD
    ))%>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame &
             had_IHD == F,
           .preserve = T) %>%
    summarise(
      first_diagnosis = unique(first_diagnosis),
      tot_allowed_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),
      office_visit_count = sum(na.rm = T, is_office_visit),
      office_visit_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_office_visit),
      cardiology_visit_count = sum(na.rm = T, is_cardiology_office_vist),
      distinct_clinicians = length(unique(PRF_PHYSN_NPI)),
      distinct_cardiologists = length(.[is_by_cardiologist, unique(PRF_PHYSN_NPI)]),
      distinct_primary_care_physicians = length(.[is_by_primary_care_physician, unique(PRF_PHYSN_NPI)]),
      distinct_surgeons = length(.[is_by_surgeon, unique(PRF_PHYSN_NPI)]),
      distinct_other_specialties= length(.[is_by_surgeon==F &
                                           is_by_cardiologist==F &
                                           is_by_primary_care_physician==F
                                           , unique(PRF_PHYSN_NPI)]),
      catheterization_count = sum(na.rm = T, is_catheterization),
      catheterization_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_catheterization),
      ecg_count = sum(na.rm = T, is_ecg),
      ecg_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_ecg),
      cardiac_ct_count = sum(na.rm = T, is_cardiac_ct),
      cardiac_ct_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_ct),
      cardiac_mri_count = sum(na.rm = T, is_cardiac_mri),
      cardiac_mri_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_mri),
      stress_test_count = sum(na.rm = T, is_stress_test),
      stress_test_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_stress_test),
      echocardiography_count = sum(na.rm = T, is_echocardiography),
      echocardiography_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_echocardiography),
      angioplasty_count = sum(na.rm = T, is_angioplasty),
      angioplasty_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_angioplasty),
      CABG_count = sum(na.rm = T, is_CABG),
      CABG_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_CABG),
      stable_angina = sum(is_stable_angina, na.rm = T) > 0,
      unstable_angina = sum(is_unstable_angina, na.rm = T) > 0,
      MI = sum(is_MI, na.rm = T) > 0,
      MI_date = head(.[is_MI,date],1),
      cardiac_arrest = sum(is_cardiac_arrest, na.rm = T) > 0,
      stroke = sum(is_stroke, na.rm = T) > 0,
      hypertension = sum(is_hypertension, na.rm = T) > 0,
      copd = sum(is_copd, na.rm = T) > 0,
      obesity = sum(is_obesity, na.rm = T) > 0,
      depression = sum(is_depression, na.rm = T) > 0,
      diabetes = sum(is_diabetes, na.rm = T) > 0,
      icd_9_pure = ifelse(prod(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, F, T),
      icd_10_pure = ifelse(sum(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, T, F),
      catheterization_date = head(.[is_catheterization,date],1),
      angioplasty_date = head(.[is_angioplasty,date],1),
      catheterization_doc_NPI = head(.[is_catheterization,PRF_PHYSN_NPI],1),
      angioplasty_doc_NPI = head(.[is_angioplasty,PRF_PHYSN_NPI],1),
      diagnosing_doc_NPI = head(.[eval(parse(text = paste("is_", diagnosis, sep = ""))),
                                                        PRF_PHYSN_NPI],1)
    ) %>%
    group_by(DESY_SORT_KEY) %>%
    mutate(
      year_first_diagnosed=year(first_diagnosis),
      time_difference_angio_cath = angioplasty_date-catheterization_date,
      angioplasty_with_cath = angioplasty_date-catheterization_date==0,
      angioplasty_during_year_after_cath = angioplasty_date-catheterization_date<=365,
    )%>%
    as.data.table()
}


summary = summarise_expenditures_carrier(yearly_patient_conditions_carrier , diagnosis = "stable_angina")
head(summary)


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath
<int>,<date>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<date>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<chr>,<chr>,<chr>,<dbl>,<int>,<lgl>,<lgl>
100007109,2013-12-26,134.60,1,114.62,0,1,0,1,0,0,0,0.00,1,19.98,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1922185792,2013,NA,NA,NA
100008015,2013-02-18,11926.19,34,1502.75,25,24,2,5,4,13,0,0.00,4,56.99,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1205894102,2013,NA,NA,NA
100016517,2013-01-06,8860.59,11,678.55,2,19,5,3,0,11,1,147.27,5,40.00,0,0,0,0,2,35.40,2,122.52,1,563.99,0,0.00,TRUE,TRUE,FALSE,NA,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,2013-06-26,2013-06-26,1881896223,1881896223,1447475637,2013,0,TRUE,TRUE
100022471,2013-02-28,6440.03,11,1080.50,3,10,1,0,1,8,0,0.00,0,0.00,0,0,0,0,1,72.42,1,202.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1548332539,2013,NA,NA,NA
100025905,2013-04-02,21060.23,0,0.00,0,76,18,5,4,49,5,365.55,21,180.81,0,0,0,0,0,0.00,2,0.00,0,0.00,2,2330.88,TRUE,TRUE,TRUE,2013-04-17,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,2013-04-02,NA,1205067220,NA,1659363760,2013,NA,NA,NA
100036059,2013-04-08,7307.08,15,1609.57,0,24,3,2,0,19,0,0.00,5,33.16,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1578554481,2013,NA,NA,NA


In [43]:
mean(summary$obesity,na.rm = T)

[1] 0.01486014

In [22]:
summary[obesity==T]

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,MI_date,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath
<int>,<date>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<date>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<chr>,<chr>,<chr>,<dbl>,<int>,<lgl>,<lgl>
100231459,2013-01-02,2709.52,22,1604.80,4,9,1,1,2,5,0,0.00,0,0.00,0,0,0,0,1,71.71,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1760477269,2013,NA,NA,NA
100414607,2013-06-10,11565.45,19,2380.43,0,17,5,4,2,6,1,197.80,16,234.36,0,0,0,0,1,79.26,2,403.71,3,1762.28,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,2013-07-16,2013-07-16,1376572933,1376572933,1932112158,2013,0,TRUE,TRUE
104818051,2013-07-22,1997.57,9,936.26,1,8,1,1,0,6,0,0.00,1,19.58,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1326088352,2013,NA,NA,NA
105386157,2013-01-09,2363.04,15,1408.22,4,18,3,3,5,7,0,0.00,6,95.48,0,0,0,0,2,37.07,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1881789253,2013,NA,NA,NA
106340709,2013-11-06,5122.15,2,221.50,0,21,4,2,1,14,2,410.38,5,39.10,0,0,0,0,0,0.00,2,98.67,0,0.00,1,1729.73,TRUE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,2013-11-06,NA,1457464471,NA,1457464471,2013,NA,NA,NA
106422747,2013-02-27,9076.97,25,2309.81,3,23,1,1,6,15,0,0.00,2,36.74,0,0,0,0,0,0.00,1,185.85,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1275516999,2013,NA,NA,NA
106526533,2013-07-03,7963.80,0,0.00,0,6,1,0,0,5,0,0.00,1,7.88,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1194763375,2013,NA,NA,NA
106705343,2013-05-21,1097.72,8,466.82,1,8,1,2,0,5,0,0.00,0,0.00,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1477635027,2013,NA,NA,NA
109474603,2013-04-16,16728.40,17,1806.07,2,48,3,10,3,32,0,0.00,4,36.52,0,0,0,0,0,0.00,0,0.00,0,0.00,0,0.00,TRUE,FALSE,FALSE,NA,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1932291648,2013,NA,NA,NA


#### Summarizing cardiology related patient data 

In [7]:
add_cardiology_related_expenditures_carrier = function(data, summary_data, time_frame = 365, diagnosis){
  
  data%>%
    group_by(DESY_SORT_KEY) %>%
    filter(sum(eval(parse(
      text = paste("is_", diagnosis, sep = "")
    )), na.rm = T) == T) %>%
    mutate(first_diagnosis = min(date[eval(parse(text = paste("is_", diagnosis, sep = ""))) ==
                                        T]), na.rm = T) %>%
    mutate( had_IHD = (
      date - first_diagnosis < 0 &
        first_diagnosis - date < time_frame &
        is_IHD
    ))%>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame &
             had_IHD == F &
             is_cardiology_related,
           .preserve = T) %>%
    summarise(
      tot_allowed_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),
      office_visit_count_cardiology_related = sum(na.rm = T, is_office_visit),
      office_visit_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_office_visit),
      cardiology_visit_count_cardiology_related = sum(na.rm = T, is_cardiology_office_vist),
      distinct_clinicians_cardiology_related = length(unique(PRF_PHYSN_NPI)),
      distinct_cardiologists_cardiology_related = length(.[is_by_cardiologist, unique(PRF_PHYSN_NPI)]),
      distinct_primary_care_physicians_cardiology_related = length(.[is_by_primary_care_physician, unique(PRF_PHYSN_NPI)]),
      distinct_surgeons_cardiology_related = length(.[is_by_surgeon, unique(PRF_PHYSN_NPI)]),
      distinct_other_specialties_cardiology_related = length(.[is_by_surgeon==F &
                                           is_by_cardiologist==F &
                                           is_by_primary_care_physician==F
                                           , unique(PRF_PHYSN_NPI)]),
      catheterization_count_cardiology_related = sum(na.rm = T, is_catheterization),
      catheterization_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_catheterization),
      ecg_count_cardiology_related = sum(na.rm = T, is_ecg),
      ecg_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_ecg),
      cardiac_ct_count_cardiology_related = sum(na.rm = T, is_cardiac_ct),
      cardiac_ct_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_ct),
      cardiac_mri_count_cardiology_related = sum(na.rm = T, is_cardiac_mri),
      cardiac_mri_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_cardiac_mri),
      stress_test_count_cardiology_related = sum(na.rm = T, is_stress_test),
      stress_test_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_stress_test),
      echocardiography_count_cardiology_related = sum(na.rm = T, is_echocardiography),
      echocardiography_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_echocardiography),
      angioplasty_count_cardiology_related = sum(na.rm = T, is_angioplasty),
      angioplasty_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_angioplasty),
      CABG_count_cardiology_related = sum(na.rm = T, is_CABG),
      CABG_cost_carrier_cardiology_related = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_CABG),
    )%>%
    left_join(summary_data, . , by = "DESY_SORT_KEY")%>%
    as.data.table()
}


summary_with_cardiology_related = add_cardiology_related_expenditures_carrier(yearly_patient_conditions_carrier, summary , diagnosis = "unstable_angina")
head(summary_with_cardiology_related)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related
<int>,<date>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<chr>,<chr>,<chr>,<dbl>,<int>,<lgl>,<lgl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>
100006149,2013-05-30,2735.95,10,1073.40,1,12,1,1,2,8,0,0.00,1,18.37,0,0,0,0,0,0.00,0,0.00,0,0,0,0,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1043214950,2013,NA,NA,NA,232.45,2,214.08,1,2,1,1,0,0,0,0.00,1,18.37,0,0,0,0,0,0.00,0,0.00,0,0,0,0
100010573,2013-04-13,6381.33,16,938.35,0,30,3,5,2,20,0,0.00,6,54.18,0,0,0,0,2,39.29,1,68.15,0,0,0,0,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1720259211,2013,NA,NA,NA,2489.84,10,459.75,0,12,2,4,0,6,0,0.00,3,27.09,0,0,0,0,2,39.29,0,0.00,0,0,0,0
100012125,2013-04-12,3144.88,15,1426.05,1,9,2,2,1,4,1,331.11,1,8.43,0,0,0,0,0,0.00,0,0.00,0,0,0,0,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,2013-04-12,NA,1033150701,NA,1033150701,2013,NA,NA,NA,450.60,1,111.06,1,2,2,0,0,0,1,331.11,1,8.43,0,0,0,0,0,0.00,0,0.00,0,0,0,0
100029887,2013-06-24,3850.90,8,603.77,0,20,4,1,4,11,0,0.00,1,8.57,0,0,0,0,0,0.00,1,65.54,0,0,0,0,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,NA,NA,NA,NA,1669439857,2013,NA,NA,NA,567.26,1,151.68,0,4,3,0,0,1,0,0.00,1,8.57,0,0,0,0,0,0.00,0,0.00,0,0,0,0
100033263,2013-06-28,8840.55,0,0.00,0,41,8,4,3,26,1,167.24,6,53.06,0,0,0,0,0,0.00,5,272.60,0,0,0,0,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,2013-08-20,NA,1417941683,NA,1972700854,2013,NA,NA,NA,2926.15,0,0.00,0,17,7,1,0,9,1,167.24,2,18.06,0,0,0,0,0,0.00,4,204.45,0,0,0,0
100035075,2013-04-06,3262.86,1,75.00,0,14,2,3,3,6,0,0.00,1,8.39,0,0,0,0,0,0.00,1,63.58,0,0,0,0,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,NA,NA,NA,NA,1023062106,2013,NA,NA,NA,329.22,0,0.00,0,4,0

### Add outpatient costs

In [8]:
outpatient_patient_characteristics=function(outpatient_revenue_center_data){
  
  #requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)
  
  outpatient_revenue_center_data %>%
    mutate(
      is_office_vist =HCPCS_CD %in% office_visit_codes,
      is_catheterization = HCPCS_CD %in% angio_codes,
      is_ecg = HCPCS_CD %in% ecg_codes,
      is_cardiac_ct = HCPCS_CD %in% cardiac_ct_codes,
      is_cardiac_mri = HCPCS_CD %in% cardiac_mri_codes,
      is_stress_test = HCPCS_CD %in% stress_test_codes,
      is_echocardiography = HCPCS_CD %in% echocardiography_codes,
      is_angioplasty = HCPCS_CD %in% angioplasty_codes,
      is_CABG = HCPCS_CD %in% CABG_codes,
      is_cardiology_related = if_else(
        PRNCPAL_DGNS_VRSN_CD == 0,
        substr(PRNCPAL_DGNS_CD, 0, 1) == "I",
        if_else(
          PRNCPAL_DGNS_VRSN_CD == 9,
          as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))>=399 & 
          as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))<=459 ,NA))
    ) %>%
    as.data.table()
}
                                      
                                      

outpatient_cost_adder=function(outpatient_and_revenue_center_data, summary_data, time_frame=365){
  
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  data=outpatient_patient_characteristics(outpatient_and_revenue_center_data)%>%as.data.table()
  data=right_join(data,
                  summary_data[,.(DESY_SORT_KEY,first_diagnosis)],by="DESY_SORT_KEY")%>%as.data.table()
  
  result=data%>%
  filter(date - first_diagnosis >= 0 &
         date - first_diagnosis < time_frame 
        ) %>%
  group_by(DESY_SORT_KEY) %>%
  summarise(
    office_visit_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_office_vist),
    catheterization_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_catheterization),
    ecg_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_ecg),
    cardiac_ct_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_cardiac_ct),
    cardiac_mri_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_cardiac_mri),
    stress_test_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_stress_test),
    echocardiography_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_echocardiography),
    angioplasty_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_angioplasty),
    CABG_cost_outpatient = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_CABG),
    office_visit_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_office_vist * is_cardiology_related),
    catheterization_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_catheterization * is_cardiology_related),
    ecg_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_ecg * is_cardiology_related),
    cardiac_ct_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_cardiac_ct * is_cardiology_related),
    cardiac_mri_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_cardiac_mri * is_cardiology_related),
    stress_test_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_stress_test * is_cardiology_related),
    echocardiography_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_echocardiography * is_cardiology_related),
    angioplasty_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_angioplasty * is_cardiology_related),
    CABG_cost_outpatient_cardiology_related = sum(na.rm = T, REV_CNTR_TOT_CHRG_AMT * is_CABG * is_cardiology_related)
    )%>%
  left_join(summary_data,.,by="DESY_SORT_KEY")%>%
  mutate_if(is.double,~replace(., is.na(.), 0))%>%
  mutate(
    office_visit_cost = office_visit_cost_outpatient+office_visit_cost_carrier,
    catheterization_cost = catheterization_cost_outpatient+catheterization_cost_carrier,
    ecg_cost = ecg_cost_outpatient+ecg_cost_carrier,
    cardiac_ct_cost = cardiac_ct_cost_outpatient+cardiac_ct_cost_carrier,
    cardiac_mri_cost = cardiac_mri_cost_outpatient+cardiac_mri_cost_carrier,
    stress_test_cost = stress_test_cost_outpatient+stress_test_cost_carrier,
    echocardiography_cost = echocardiography_cost_outpatient+echocardiography_cost_carrier,
    angioplasty_cost = angioplasty_cost_outpatient+angioplasty_cost_carrier,
    CABG_cost = CABG_cost_outpatient+CABG_cost_carrier,
    office_visit_cost_cardiology_related = office_visit_cost_outpatient_cardiology_related+office_visit_cost_carrier_cardiology_related,
    catheterization_cost_cardiology_related = catheterization_cost_outpatient_cardiology_related+catheterization_cost_carrier_cardiology_related,
    ecg_cost_cardiology_related = ecg_cost_outpatient_cardiology_related+ecg_cost_carrier_cardiology_related,
    cardiac_ct_cost_cardiology_related = cardiac_ct_cost_outpatient_cardiology_related+cardiac_ct_cost_carrier_cardiology_related,
    cardiac_mri_cost_cardiology_related = cardiac_mri_cost_outpatient_cardiology_related+cardiac_mri_cost_carrier_cardiology_related,
    stress_test_cost_cardiology_related = stress_test_cost_outpatient_cardiology_related+stress_test_cost_carrier_cardiology_related,
    echocardiography_cost_cardiology_related = echocardiography_cost_outpatient_cardiology_related+echocardiography_cost_carrier_cardiology_related,
    angioplasty_cost_cardiology_related = angioplasty_cost_outpatient_cardiology_related+angioplasty_cost_carrier_cardiology_related,
    CABG_cost_cardiology_related = CABG_cost_outpatient_cardiology_related+CABG_cost_carrier_cardiology_related
  )%>%
  group_by(DESY_SORT_KEY)%>%
  mutate(
    tot_cheap_prcdr_cost = sum(
        stress_test_cost,
        echocardiography_cost,
        office_visit_cost,na.rm = T),
    tot_expensive_prcdr_cost = sum(
        catheterization_cost,
        cardiac_ct_cost,
        angioplasty_cost
        ,na.rm = T),
    tot_cheap_prcdr_cost_cardiology_related = sum(
        stress_test_cost_cardiology_related,
        echocardiography_cost_cardiology_related,
        office_visit_cost_cardiology_related,na.rm = T),
    tot_expensive_prcdr_cost_cardiology_related = sum(
        catheterization_cost_cardiology_related,
        cardiac_ct_cost_cardiology_related,
        angioplasty_cost_cardiology_related
        ,na.rm = T),
    
    tot_cheap_prcdr_count = sum(
        stress_test_count,
        echocardiography_count,
        office_visit_count,na.rm = T),
    tot_expensive_prcdr_count = sum(
        catheterization_count,
        cardiac_ct_count,
        angioplasty_count
        ,na.rm = T),
    tot_cheap_prcdr_count_cardiology_related = sum(
        stress_test_count_cardiology_related,
        echocardiography_count_cardiology_related,
        office_visit_count_cardiology_related,na.rm = T),
    tot_expensive_prcdr_count_cardiology_related = sum(
        catheterization_count_cardiology_related,
        cardiac_ct_count_cardiology_related,
        angioplasty_count_cardiology_related
        ,na.rm = T)
  )%>%
  as.data.table()
  return(result)
}

summary_with_outpatient=outpatient_cost_adder(outpatient_and_revenue_center_data,summary_with_cardiology_related)
head(summary_with_outpatient)

Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”


DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related
<int>,<date>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<chr>,<chr>,<chr>,<dbl>,<int>,<lgl>,<lgl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<i

### Patient characteristics (MBSF)

Here, we will look for patient characteristics that might get them excluded from our analyses. Things like age, state, etc.
I will use foreach parallel computing.

In [9]:
twelve_months_after=function(data,colname){

  z=foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        paste(colname,
              format(data[p,"first_diagnosis"] %m+% months(a),"%m"),"_",
              format(data[p,"first_diagnosis"] %m+% months(a),"%Y"),sep="")
      }
  
  y=foreach(p = 1:nrow(data), .combine = rbind) %:%
      foreach (a = 0:11, .combine = cbind) %dopar% {
        as.numeric(format(data[p,"first_diagnosis"] %m+% months(a),"%Y"))
      }

  return(list(z,y))
}

monthly_characteristics_finder=function(data,colname){
  
  require(dtplyr)
  require(tidyverse)

  z=twelve_months_after(data,colname)[[1]] 
  y=twelve_months_after(data,colname)[[2]] 
  
  result = foreach (p = 1:nrow(data), .combine = rbind) %:%
      foreach (i = 1:12, .combine = cbind) %dopar% {
        if(y[p,i]<2021){
          a=as.character(z[p,i])
          b=data[p,a]
        }
        else{
          b=NA
        }
        b
      }
  return(result)
}


patient_year_of_diagnosis_characteristic_finder = function(data,var_name){
  
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  
  var=c(paste(var_name,"_",data[,"year_first_diagnosed"],sep=""))
  result=foreach (a = 1:nrow(data),.combine = rbind) %dopar%{
    data[a,var[a]]
    }
  
  return(result)
}

add_patient_characteristics = function(mbsf_data,summary_data){
  
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  data = left_join(summary_data,mbsf_data,by="DESY_SORT_KEY") %>% as.data.frame()
  
  result=data%>%
    mutate(   
      state_code_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(.,"STATE_CODE"),
      county_code_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(.,"COUNTY_CODE"),
      sex_code_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(.,"SEX_CODE"),
      race_code_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(.,"RACE_CODE"),
      age_at_diagnosis=patient_year_of_diagnosis_characteristic_finder(.,"AGE"),
      ENTITLEMENT_BUY_IN_IND_sum=
      rowSums(monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")==3 
              | monthly_characteristics_finder(data,"ENTITLEMENT_BUY_IN_IND")=="C"
              ,na.rm=T),
      HMO_INDICATOR_sum=
      rowSums(monthly_characteristics_finder(data,"HMO_INDICATOR")==0
              ,na.rm=T),
      died_in_one_year_after_diagnosis=date_of_death_collapsed<first_diagnosis+365,
      died_in_two_years_after_diagnosis=date_of_death_collapsed<first_diagnosis+730,
      date_difference_diagnosis_death=date_of_death_collapsed-first_diagnosis
          ) %>% 
  as.data.table()
  
  result[is.na(died_in_one_year_after_diagnosis), died_in_one_year_after_diagnosis:=FALSE]
  result[is.na(died_in_two_years_after_diagnosis), died_in_two_years_after_diagnosis:=FALSE]

  result[as.IDate("20191231", "%Y%m%d")<first_diagnosis 
                , died_in_one_year_after_diagnosis:=NA]

  result[as.IDate("20181231", "%Y%m%d")<first_diagnosis 
                , died_in_two_years_after_diagnosis:=NA]
  return(result)
}


summary_with_patient_characteristics=add_patient_characteristics(mbsf_data,summary_with_outpatient)
head(summary_with_patient_characteristics)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

#### Single core MBSF
These are the same functions as above but without parallelization.

### Adding comorbidities

Here, I will add patient comorbidity scores.


In [12]:
#library(remotes)
#remotes::install_github("ellessenne/comorbidity@0.5.3")

checkmate (2.0.0 -> 2.1.0) [CRAN]


Installing 1 packages: checkmate




The downloaded binary packages are in
	/var/folders/gr/r594hq9j4klg4g_b0qwqkpdc0000gp/T//RtmpLN6PEQ/downloaded_packages
✔  checking for file ‘/private/var/folders/gr/r594hq9j4klg4g_b0qwqkpdc0000gp/T/RtmpLN6PEQ/remotes26b56d7b6674/ellessenne-comorbidity-c30f002/DESCRIPTION’ (572ms)
─  preparing ‘comorbidity’: (610ms)
✔  checking DESCRIPTION meta-information
─  installing the package to process help pages
─  saving partial Rd database (2.4s)
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘comorbidity_0.5.3.tar.gz’
   


In [10]:
add_comorbidity=function(data, summary_data, time_frame = 365){

  require(comorbidity)
  
  #adding comorbidities
  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY",
                                "first_diagnosis",
                                "icd_9_pure",
                                "icd_10_pure")], by = "DESY_SORT_KEY") %>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame) %>%
    as.data.table()
  
  
  comorbidity_icd_9 = comorbidity_and_phys_data %>%
    subset(icd_9_pure == T)
  
  if (nrow(comorbidity_icd_9) != 0) {
    comorbidity_icd_9 = comorbidity(
      as.data.table(comorbidity_icd_9),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd9",
      assign0 = T
    )
  }
  else {
    comorbidity_icd_9 = data.table(
      DESY_SORT_KEY = NA,
      score = NA,
      index = NA,
      wscore = NA,
      windex = NA
    )
  }
  
  comorbidity_icd_10 = comorbidity_and_phys_data %>%
    subset(icd_10_pure == T)
  
  if (nrow(comorbidity_icd_10) != 0) {
    comorbidity_icd_10 = comorbidity(
      as.data.table(comorbidity_icd_10),
      id = "DESY_SORT_KEY",
      code = "LINE_ICD_DGNS_CD",
      score = "charlson",
      icd = "icd10",
      assign0 = T
    )
  }
  else {
    comorbidity_icd_10 = data.table(
      DESY_SORT_KEY = NA,
      score = NA,
      index = NA,
      wscore = NA,
      windex = NA
    )
  }
 
  comorbidity_all=rbind(
    comorbidity_icd_9[,c("DESY_SORT_KEY","score","index","wscore","windex")]
    ,comorbidity_icd_10[,c("DESY_SORT_KEY","score","index","wscore","windex")]
  )
  result = left_join(summary_data,
                     comorbidity_all,
                     by="DESY_SORT_KEY",) %>% as.data.table()

}

summary_with_patient_characteristics_comorbidity=add_comorbidity(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics)
head(summary_with_patient_characteristics_comorbidity)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

### Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [11]:
#adding most common physicians
add_patient_NPI=function(data, summary_data, time_frame = 365){

  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY",
                                "first_diagnosis",
                                "icd_9_pure",
                                "icd_10_pure")], by = "DESY_SORT_KEY") %>%
    filter(date - first_diagnosis >= 0 &
             date - first_diagnosis < time_frame) %>%
    as.data.table()
  
  patient_NPI_count_finder = function(data) {
    result = data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes)%>%
      group_by(DESY_SORT_KEY, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit,na.rm=T)) %>%
      filter(n>0)%>%
      arrange(.by_group = T, desc(n))
  }
  
  patient_NPI_counts = patient_NPI_count_finder(comorbidity_and_phys_data)
  
  patient_NPI_counts = left_join(patient_NPI_counts,
                                 distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]), by ="PRF_PHYSN_NPI")
  
  find_most_common = function(data) {
    data %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  find_most_common_by_specialty = function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  most_common_physician = find_most_common(patient_NPI_counts)
  #primary care = 01:general practice/ family practice:08/ internal medicine:11/ geriatrics:38
  most_common_primary_care_physician = find_most_common_by_specialty(patient_NPI_counts,
                                                                     specialty_code = c("01", "08", "11", "38"))
  most_common_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = c("06","C3"))
  #most_common_interventional_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = "C3")
  
  most_common_physician = data.frame(most_common_physician) %>%
    rename_with( ~ paste0("most_common_physician_", .x))
  most_common_primary_care_physician = data.frame(most_common_primary_care_physician) %>%
    rename_with( ~ paste0("most_common_primary_care_physician_", .x))
  most_common_cardiologists = data.frame(most_common_cardiologists) %>%
    rename_with( ~ paste0("most_common_cardiologist_", .x))
  #most_common_interventional_cardiologists = data.frame(most_common_interventional_cardiologists) %>%
   # rename_with( ~ paste0("most_common_interventional_cardiologist_", .x))
  
  summary_data = left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_cardiologists,
    by = c("DESY_SORT_KEY" = "most_common_cardiologist_DESY_SORT_KEY")
  )%>%
  as.data.table()
  
  #summary_data = left_join(
  #  summary_data,
  #  most_common_interventional_cardiologists,
  #  by = c("DESY_SORT_KEY" = "most_common_interventional_cardiologist_DESY_SORT_KEY")
  #)%>%
  #  as.data.table()

  #summary_data[, year_first_diagnosis := lubridate::year(first_diagnosis)]%>%
    #as.data.table()

}

#summary_with_npi=add_patient_NPI(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics_comorbidity)
#head(summary_with_npi)


#### Most common physicians for each patient at 2013

In [12]:
#adding most common physicians
add_patient_NPI_2013=function(data, summary_data){

  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY",
                                "first_diagnosis",
                                "icd_9_pure",
                                "icd_10_pure")], by = "DESY_SORT_KEY") %>%
    filter(date < as.IDate("2014-01-01")) %>%
    as.data.table()
  
  patient_NPI_count_finder = function(data) {
    result = data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes)%>%
      group_by(DESY_SORT_KEY, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit,na.rm=T)) %>%
      filter(n>0)%>%
      arrange(.by_group = T, desc(n))
  }
  
  patient_NPI_counts = patient_NPI_count_finder(comorbidity_and_phys_data)
  
  patient_NPI_counts = left_join(patient_NPI_counts,
                                 distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]), by ="PRF_PHYSN_NPI")
  
  find_most_common = function(data) {
    data %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  find_most_common_by_specialty = function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  most_common_physician = find_most_common(patient_NPI_counts)
  #primary care = 01:general practice/ family practice:08/ internal medicine:11/ geriatrics:38
  most_common_primary_care_physician = find_most_common_by_specialty(patient_NPI_counts,
                                                                     specialty_code = c("01", "08", "11", "38"))
  most_common_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = c("06","C3"))
  #most_common_interventional_cardiologists = find_most_common_by_specialty(patient_NPI_counts, specialty_code = "C3")
  
  most_common_physician = data.frame(most_common_physician) %>%
    rename_with( ~ paste0("most_common_physician_2013_", .x))
  most_common_primary_care_physician = data.frame(most_common_primary_care_physician) %>%
    rename_with( ~ paste0("most_common_primary_care_physician_2013_", .x))
  most_common_cardiologists = data.frame(most_common_cardiologists) %>%
    rename_with( ~ paste0("most_common_cardiologist_2013_", .x))
  #most_common_interventional_cardiologists = data.frame(most_common_interventional_cardiologists) %>%
   # rename_with( ~ paste0("most_common_interventional_cardiologist_", .x))
  
  summary_data = left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_2013_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_2013_DESY_SORT_KEY")
  )
  summary_data = left_join(
    summary_data,
    most_common_cardiologists,
    by = c("DESY_SORT_KEY" = "most_common_cardiologist_2013_DESY_SORT_KEY")
  )%>%
  as.data.table()
  
  #summary_data = left_join(
  #  summary_data,
  #  most_common_interventional_cardiologists,
  #  by = c("DESY_SORT_KEY" = "most_common_interventional_cardiologist_DESY_SORT_KEY")
  #)%>%
  #  as.data.table()

  #summary_data[, year_first_diagnosis := lubridate::year(first_diagnosis)]%>%
    #as.data.table()

}

#summary_with_npi_and_2013_npi=add_patient_NPI_2013(data = carrier_data_all_years, summary_data = summary_with_npi)
#head(summary_with_npi_and_2013_npi)


## Combining all results

Here, I will combine the results of all functions above.

In [13]:
yearly_calculator = function(data,mbsf_data,revenue_center_outpatient_data,diagnosis){

  require(tidyverse)
  require(lubridate)
  require(dtplyr)

  yearly_patient_conditions_carrier =
  data %>%
  yearly_calculator_patient_conditions()%>%
  as.data.table()
  
  result = 
  summarise_expenditures_carrier(yearly_patient_conditions_carrier , diagnosis = diagnosis) %>%
  add_cardiology_related_expenditures_carrier(yearly_patient_conditions_carrier, summary_data = . , diagnosis = diagnosis)%>%
  outpatient_cost_adder(revenue_center_outpatient_data, summary_data = .)%>%
  add_patient_characteristics(mbsf_data = mbsf_data, summary_data = .)%>%
  add_comorbidity(data = data, summary_data = .) %>%
  add_patient_NPI(data = data, summary_data = .) %>%
  add_patient_NPI_2013(data=data , summary_data = .)%>%
  as.data.table()
  
  return(result)

}

### Stable angina

In [48]:
yearly_calcualtions_carrier_stable_angina=
yearly_calculator(data=carrier_data_all_years,
                  mbsf_data=mbsf_data,
                  revenue_center_outpatient_data=outpatient_and_revenue_center_data,
                  diagnosis="stable_angina")

Warning message in fifelse(LINE_ICD_DGNS_VRSN_CD == 9, as.numeric(substr(LINE_ICD_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(LINE_ICD_DGNS_VRSN_CD == 9, as.numeric(substr(LINE_ICD_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [49]:
head(yearly_calcualtions_carrier_stable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

In [ ]:
write_fst(yearly_calcualtions_carrier_stable_angina,
          "results_apr/yearly_calcualtions_carrier_stable_angina.fst")

### Unstable angina

In [50]:
yearly_calcualtions_carrier_unstable_angina=
yearly_calculator(data=carrier_data_all_years,
                  mbsf_data=mbsf_data,
                  revenue_center_outpatient_data=outpatient_and_revenue_center_data,
                  diagnosis="unstable_angina")

Warning message in fifelse(LINE_ICD_DGNS_VRSN_CD == 9, as.numeric(substr(LINE_ICD_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(LINE_ICD_DGNS_VRSN_CD == 9, as.numeric(substr(LINE_ICD_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [51]:
head(yearly_calcualtions_carrier_unstable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

In [ ]:
write_fst(yearly_calcualtions_carrier_unstable_angina,
          "results_apr/yearly_calcualtions_carrier_unstable_angina.fst")

## Reading the results for patient yearly carrier calculations
This is if I have already done the steps above.

In [46]:
yearly_calcualtions_carrier_stable_angina = 
read_fst(path = "results_mar/yearly_calcualtions_carrier_stable_angina.fst"
        ,as.data.table = T)

yearly_calcualtions_carrier_unstable_angina = 
read_fst(path = "results_mar/yearly_calcualtions_carrier_unstable_angina.fst"
        ,as.data.table = T)

#physician_integration_stats = 
#read_fst("results_feb/physician_integration_stats.fst"
#        ,as.data.table = T)

head(yearly_calcualtions_carrier_stable_angina)
head(yearly_calcualtions_carrier_unstable_angina)
#head(physician_integration_stats)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,⋯,ENTITLEMENT_BUY_IN_IND02_2017,ENTITLEMENT_BUY_IN_IND03_2017,ENTITLEMENT_BUY_IN_IND04_2017,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_2017,HMO_INDICATOR01_2017,HMO_INDICATOR02_2017,HMO_INDICATOR03_2017,HMO_INDICATOR04_2017,HMO_INDICATOR05_2017,HMO_INDICATOR06_2017,HMO_INDICATOR07_2017,HMO_INDICATOR08_2017,HMO_INDICATOR09_2017,HMO_INDICATOR10_2017,HMO_INDICATOR11_2017,HMO_INDICATOR12_2017,VALID_DATE_OF_DEATH_SWITCH_2017,DATE_OF_DEATH_2017,REFERENCE_YEAR_2018

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,tot_cheap_prcdr,tot_expensive_prcdr,year_first_diagnosed,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLEMENT_BUY_IN_IND02_2014,ENTITLEMENT_BUY_IN_IND03_2014,ENTITLEMENT_BUY_IN_IND04_2014,ENTITLEMENT_BUY_IN_IND05_2014,ENTITLEMENT_BUY_IN_IND06_2014,ENTITLEMENT_BUY_IN_IND07_2014,ENTITLEMENT_BUY_IN_IND08_2014,ENTITLEMENT_BUY_IN_IND09_2014,ENTITLEMENT_BUY_IN_IND10_2014,ENTITLEMENT_BUY_IN_IND11_2014,ENTITLEMENT_BUY_IN_IND12_2014,HMO_INDICATOR01_2014,HMO_INDICATOR02_2014,HMO_INDICATOR03_2014,HMO_INDICATOR04_2014,HMO_INDICATOR05_2014,HMO_INDICATOR06_2014,HMO_INDICATOR07_2014,HMO_INDICATOR08_2014,HMO_INDICATOR09_2014,HMO_INDICATOR10_2014,HMO_INDICATOR11_2014,HMO_INDICATOR12_2014,VALID_DATE_OF_DEATH_SWITCH_2014,DATE_OF_DEATH_2014,REFERENCE_YEAR_2015,STATE_CODE_2015,COUNTY_CODE_2015,SEX_CODE_2015,RACE_CODE_2015,AGE_2015,ORIG_REASON_FOR_ENTITLEMENT_2015,CURR_REASON_FOR_ENTITLEMENT_2015,ENTITLEMENT_BUY_IN_IND01_2015,ENTITLEMENT_BUY_IN_IND02_2015,ENTITLEMENT_BUY_IN_IND03_2015,ENTITLEMENT_BUY_IN_IND04_2015,ENTITLEMENT_BUY_IN_IND05_2015,ENTITLEMENT_BUY_IN_IND06_2015,ENTITLEMENT_BUY_IN_IND07_2015,ENTITLEMENT_BUY_IN_IND08_2015,ENTITLEMENT_BUY_IN_IND09_2015,ENTITLEMENT_BUY_IN_IND10_2015,ENTITLEMENT_BUY_IN_IND11_2015,ENTITLEMENT_BUY_IN_IND12_2015,⋯,ENTITLEMENT_BUY_IN_IND02_2017,ENTITLEMENT_BUY_IN_IND03_2017,ENTITLEMENT_BUY_IN_IND04_2017,ENTITLEMENT_BUY_IN_IND05_2017,ENTITLEMENT_BUY_IN_IND06_2017,ENTITLEMENT_BUY_IN_IND07_2017,ENTITLEMENT_BUY_IN_IND08_2017,ENTITLEMENT_BUY_IN_IND09_2017,ENTITLEMENT_BUY_IN_IND10_2017,ENTITLEMENT_BUY_IN_IND11_2017,ENTITLEMENT_BUY_IN_IND12_2017,HMO_INDICATOR01_2017,HMO_INDICATOR02_2017,HMO_INDICATOR03_2017,HMO_INDICATOR04_2017,HMO_INDICATOR05_2017,HMO_INDICATOR06_2017,HMO_INDICATOR07_2017,HMO_INDICATOR08_2017,HMO_INDICATOR09_2017,HMO_INDICATOR10_2017,HMO_INDICATOR11_2017,HMO_INDICATOR12_2017,VALID_DATE_OF_DEATH_SWITCH_2017,DATE_OF_DEATH_2017,REFERENCE_YEAR_2018

## Adding inpatient and outpatient expenditure data

Adding the data to the yearly calculations with an inner join so I will only have data for stable/unstable angina patients. I will also filter based on date in this part.


In [52]:
outpatient_data_all_years_stable_angina=inner_join(outpatient_data_all_years, yearly_calcualtions_carrier_stable_angina,by="DESY_SORT_KEY")%>%as.data.table()
intpatient_data_all_years_stable_angina=inner_join(inpatient_data_all_years, yearly_calcualtions_carrier_stable_angina,by="DESY_SORT_KEY")%>%as.data.table()

outpatient_data_all_years_unstable_angina=inner_join(outpatient_data_all_years, yearly_calcualtions_carrier_unstable_angina,by="DESY_SORT_KEY")%>%as.data.table()
intpatient_data_all_years_unstable_angina=inner_join(inpatient_data_all_years, yearly_calcualtions_carrier_unstable_angina,by="DESY_SORT_KEY")%>%as.data.table()

This will take the outpatient and inpatient data and retun the total outpatient and inpatient expenditure. It also asks for the time frame which will be compared to the date first diagnosed. I will keep using yearly here.

In [53]:
#outpatient
yearly_tot_outpatient=function(data,time_frame=365){
  data%>%
  mutate(is_cardiology_related = if_else(
    PRNCPAL_DGNS_VRSN_CD == 0,
    substr(PRNCPAL_DGNS_CD, 0, 1) == "I",
    if_else(
      PRNCPAL_DGNS_VRSN_CD == 9,
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))>=399 & 
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))<=459 ,NA)))%>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_outpatient=sum(CLM_TOT_CHRG_AMT))%>%
    as.data.table()
}

yearly_tot_outpatient_cardiology_related=function(data,time_frame=365){
  data%>%
  mutate(is_cardiology_related = if_else(
    PRNCPAL_DGNS_VRSN_CD == 0,
    substr(PRNCPAL_DGNS_CD, 0, 1) == "I",
    if_else(
      PRNCPAL_DGNS_VRSN_CD == 9,
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))>=399 & 
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))<=459 ,NA)))%>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame & is_cardiology_related)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_outpatient_cardiology_related=sum(CLM_TOT_CHRG_AMT))%>%
    as.data.table()
}

#inpatient
yearly_calculator_inpatient=function (data, time_frame=365){
  data %>%
  mutate(is_cardiology_related = if_else(
    PRNCPAL_DGNS_VRSN_CD == 0,
    substr(PRNCPAL_DGNS_CD, 0, 1) == "I",
    if_else(
      PRNCPAL_DGNS_VRSN_CD == 9,
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))>=399 & 
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))<=459 ,NA)))%>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_inpatient=sum(CLM_TOT_CHRG_AMT),
              number_of_hospitalizations=length(unique(date)[is.na(CLM_DRG_CD)==F]),
    ) %>%
    mutate(was_hospitalized=number_of_hospitalizations>0)%>%
    as.data.table()
}

#inpatient
yearly_calculator_inpatient_cardiology_related=function (data, time_frame=365){
  data %>%
  mutate(is_cardiology_related = if_else(
    PRNCPAL_DGNS_VRSN_CD == 0,
    substr(PRNCPAL_DGNS_CD, 0, 1) == "I",
    if_else(
      PRNCPAL_DGNS_VRSN_CD == 9,
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))>=399 & 
      as.numeric(substr(PRNCPAL_DGNS_CD, 0, 3))<=459 ,NA)))%>%
    filter(date>=first_diagnosis & date-first_diagnosis<time_frame & is_cardiology_related)%>%
    group_by(DESY_SORT_KEY)%>%
    summarise(tot_allowed_inpatient_cardiology_related=sum(CLM_TOT_CHRG_AMT),
              number_of_hospitalizations_cardiology_related=length(unique(date)[is.na(CLM_DRG_CD)==F]),
    ) %>%
    mutate(was_hospitalized_cardiology_related=number_of_hospitalizations_cardiology_related>0)%>%
    as.data.table()
}



outpatient_tot_yearly_stable_angina=yearly_tot_outpatient(outpatient_data_all_years_stable_angina)
inpatient_tot_yearly_stable_angina=yearly_calculator_inpatient(intpatient_data_all_years_stable_angina)

outpatient_tot_yearly_stable_angina_cardiology_related=
yearly_tot_outpatient_cardiology_related(outpatient_data_all_years_stable_angina)
inpatient_tot_yearly_stable_angina_cardiology_related=
yearly_calculator_inpatient_cardiology_related(intpatient_data_all_years_stable_angina)


outpatient_tot_yearly_unstable_angina=yearly_tot_outpatient(outpatient_data_all_years_unstable_angina)
inpatient_tot_yearly_unstable_angina=yearly_calculator_inpatient(intpatient_data_all_years_unstable_angina)

outpatient_tot_yearly_unstable_angina_cardiology_related=
yearly_tot_outpatient_cardiology_related(outpatient_data_all_years_unstable_angina)
inpatient_tot_yearly_unstable_angina_cardiology_related=
yearly_calculator_inpatient_cardiology_related(intpatient_data_all_years_unstable_angina)



Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_VRSN_CD == 9, as.numeric(substr(PRNCPAL_DGNS_CD, :
“NAs introduced by coercion”
Warning message in fifelse(PRNCPAL_DGNS_

### Adding the outpatient and inpatient results to the carrier results
I will also find the total expenditure in one year

In [54]:
#stable angina

yearly_calculations_stable_angina=left_join(yearly_calcualtions_carrier_stable_angina, outpatient_tot_yearly_stable_angina, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_stable_angina=left_join(yearly_calculations_stable_angina, inpatient_tot_yearly_stable_angina, by="DESY_SORT_KEY")%>%as.data.table()

yearly_calculations_stable_angina=
left_join(yearly_calculations_stable_angina, outpatient_tot_yearly_stable_angina_cardiology_related, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_stable_angina=
left_join(yearly_calculations_stable_angina, inpatient_tot_yearly_stable_angina_cardiology_related, by="DESY_SORT_KEY")%>%as.data.table()

#finding the total expenditure in one year
yearly_calculations_stable_angina[,total_exp:=sum(tot_allowed_carrier,tot_allowed_outpatient,tot_allowed_inpatient,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_stable_angina[is.na(number_of_hospitalizations)==T,`:=`(number_of_hospitalizations=0,was_hospitalized=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_outpatient)==T,`:=`(tot_allowed_outpatient=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_inpatient)==T,`:=`(tot_allowed_inpatient=0)]

yearly_calculations_stable_angina[,total_exp_cardiology_related:=sum(tot_allowed_carrier_cardiology_related,tot_allowed_outpatient_cardiology_related,tot_allowed_inpatient_cardiology_related,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_stable_angina[is.na(number_of_hospitalizations_cardiology_related)==T,`:=`(number_of_hospitalizations_cardiology_related=0,was_hospitalized_cardiology_related=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_outpatient_cardiology_related)==T,`:=`(tot_allowed_outpatient_cardiology_related=0)]
yearly_calculations_stable_angina[is.na(tot_allowed_inpatient_cardiology_related)==T,`:=`(tot_allowed_inpatient_cardiology_related=0)]



In [55]:
head(yearly_calculations_stable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

In [56]:
#unstable angina

yearly_calculations_unstable_angina=left_join(yearly_calcualtions_carrier_unstable_angina, outpatient_tot_yearly_unstable_angina, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_unstable_angina=left_join(yearly_calculations_unstable_angina, inpatient_tot_yearly_unstable_angina, by="DESY_SORT_KEY")%>%as.data.table()

yearly_calculations_unstable_angina=
left_join(yearly_calculations_unstable_angina, outpatient_tot_yearly_unstable_angina_cardiology_related, by="DESY_SORT_KEY")%>%as.data.table()
yearly_calculations_unstable_angina=
left_join(yearly_calculations_unstable_angina, inpatient_tot_yearly_unstable_angina_cardiology_related, by="DESY_SORT_KEY")%>%as.data.table()

#finding the total expenditure in one year
yearly_calculations_unstable_angina[,total_exp:=sum(tot_allowed_carrier,tot_allowed_outpatient,tot_allowed_inpatient,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_unstable_angina[is.na(number_of_hospitalizations)==T,`:=`(number_of_hospitalizations=0,was_hospitalized=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_outpatient)==T,`:=`(tot_allowed_outpatient=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_inpatient)==T,`:=`(tot_allowed_inpatient=0)]

yearly_calculations_unstable_angina[,total_exp_cardiology_related:=sum(tot_allowed_carrier_cardiology_related,tot_allowed_outpatient_cardiology_related,tot_allowed_inpatient_cardiology_related,na.rm = T),by=DESY_SORT_KEY]
yearly_calculations_unstable_angina[is.na(number_of_hospitalizations_cardiology_related)==T,`:=`(number_of_hospitalizations_cardiology_related=0,was_hospitalized_cardiology_related=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_outpatient_cardiology_related)==T,`:=`(tot_allowed_outpatient_cardiology_related=0)]
yearly_calculations_unstable_angina[is.na(tot_allowed_inpatient_cardiology_related)==T,`:=`(tot_allowed_inpatient_cardiology_related=0)]



In [57]:
head(yearly_calculations_unstable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

## Physician integration status

Here, I will find which physicians are integrated.


### A method of finding codes that are not exclusive to hospitals or non-hospital places
We can exclude these HCPCS codes and only include codes that are not exclusive to hospitals.

In [24]:
exclusive_hospital_code_finder = function (data,
                                           threshold=0.05,
                                           integrated_place_of_service_codes = c("19", "22"),
                                           all_place_of_service_codes = c("11", "19", "22")){
  require(dtplyr)
  require(tidyverse)
  
  result = data %>%
  filter(LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes)%>%
  group_by(HCPCS_CD) %>%
  summarise(prp_in_facility = nrow(.[LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes])/n()
           )%>%
  as.data.table
  
  exclusive_codes = result[prp_in_facility<threshold | prp_in_facility>(1-threshold),HCPCS_CD]
  

  return(exclusive_codes)
  
}

exclusive_hospital_codes=exclusive_hospital_code_finder(carrier_data_all_years)

In [25]:
exclusive_hospital_codes

[1] "00100" "00102" "00103" "00104" "00120" "00126" "00140" "00142" "00144"
  [10] "00145" "00160" "00170" "00190" "00210" "00211" "00216" "00283" "00300"
  [19] "00320" "00350" "00352" "00400" "00402" "00404" "00410" "00520" "00528"
  [28] "00530" "00532" "00534" "00537" "00541" "00542" "00560" "00562" "00566"
  [37] "00567" "00620" "00630" "00670" "00700" "00702" "00750" "00752" "00770"
  [46] "00790" "00800" "00802" "00820" "00830" "00832" "00840" "00846" "00851"
  [55] "00860" "00862" "00865" "00872" "00873" "00904" "00912" "00914" "00918"
  [64] "00920" "00938" "00942" "00944" "00952" "01112" "01202" "01210" "01214"
  [73] "01215" "01220" "01250" "01270" "01320" "01380" "01382" "01392" "01400"
  [82] "01402" "01430" "01462" "01474" "01480" "01482" "01520" "01610" "01620"
  [91] "01622" "01630" "01638" "01652" "01710" "01730" "01740" "01744" "0174T"
 [100] "01770" "01810" "01820" "01829" "01830" "01840" "01844" "01860" "0191T"
 [109] "01920" "01922" "01925" "01926" "01935" "0197T" "0198T" "0201T" "0207T"
 [118] "0238T" "0240T" "0291T" "0295T" "0296T" "0297T" "0509F" "0517F" "0518F"
 [127] "0521F" "0540F" "0556F" "10040" "10060" "1006F" "10120" "1012F" "1031F"
 [136] "1032F" "1033F" "1034F" "1035F" "1038F" "1040F" "1055F" "1091F" "11000"
 [145] "11001" "11012" "11047" "11055" "11056" "11057" "11100" "11101" "11201"
 [154] "1125F" "1126F" "11301" "11303" "11305" "11306" "11307" "11308" "1130F"
 [163] "11310" "11311" "11312" "11313" "11400" "11420" "11440" "11441" "11443"
 [172] "11444" "11446" "11463" "1159F" "11600" "11601" "1160F" "11620" "11640"
 [181] "1170F" "11719" "11720" "11721" "11730" "11732" "11740" "11750" "11755"
 [190] "1175F" "11765" "1181F" "11900" "11901" "11980" "11983" "12005" "1200F"
 [199] "12011" "12013" "12021" "12044" "1205F" "1220F" "13100" "13120" "13122"
 [208] "13131" "13151" "13153" "14020" "1490F" "1494F" "15003" "15101" "15115"
 [217] "15272" "15276" "15576" "15630" "15732" "15734" "15757" "15822" "15832"
 [226] "15850" "15851" "15860" "16025" "17000" "17003" "17004" "17110" "17111"
 [235] "17260" "17261" "17262" "17263" "17264" "17270" "17271" "17273" "17274"
 [244] "17280" "17281" "17282" "17283" "17313" "17340" "17999" "19100" "19101"
 [253] "19125" "19126" "19290" "19291" "19296" "19298" "19301" "19302" "19303"
 [262] "19307" "19316" "19318" "19328" "19330" "19340" "19342" "19350" "19370"
 [271] "19371" "19380" "2000F" "2001F" "2002F" "2010F" "2016F" "2019F" "20205"
 [280] "20206" "2021F" "20220" "20225" "2022F" "20240" "2024F" "2027F" "2029F"
 [289] "2040F" "20520" "20527" "20550" "20551" "20600" "20605" "20665" "20692"
 [298] "20902" "20910" "20912" "20926" "20930" "20936" "20974" "20979" "20985"
 [307] "21011" "21014" "21016" "21040" "21046" "21210" "21248" "21275" "21330"
 [316] "21337" "21501" "21550" "21552" "21554" "21800" "21920" "21930" "21932"
 [325] "21936" "22206" "22310" "22521" "22522" "22612" "22614" "22802" "22843"
 [334] "22903" "23040" "23120" "23130" "23405" "23410" "23412" "23420" "23430"
 [343] "23440" "23470" "23500" "23605" "23615" "23620" "23650" "23700" "23930"
 [352] "24065" "24105" "24341" "24342" "24366" "24500" "24505" "24515" "24545"
 [361] "24576" "24670" "24685" "25000" "25065" "25075" "25112" "25118" "25215"
 [370] "25240" "25246" "25301" "25310" "25360" "25390" "25391" "25447" "25530"
 [379] "25545" "25600" "25606" "25607" "25608" "25609" "25622" "25630" "26010"
 [388] "26040" "26070" "26080" "26111" "26121" "26123" "26125" "26145" "26340"
 [397] "26341" "26350" "26415" "26432" "26440" "26449" "26480" "26508" "26725"
 [406] "26770" "26841" "26850" "26860" "26951" "26952" "27093" "27095" "27130"
 [415] "27193" "27235" "27265" "27266" "27323" "27327" "27328" "27370" "27385"
 [424] "27442" "27447" "27486" "27506" "27520" "27524" "27530" "27570" "27602"
 [433] "27613" "27616" "27618" "27626" "27648" "27659" "27675" "27680" "27685"
 [442] "27687" "27691" "27695" "27698" "27760" "27766" "27780" "27792" "27808"
 [451] "27823" "27826" "27829" "27870" "27880" "27881

### A function to find integrated docs

In [26]:
#calculate and add physician integration data
#this only uses visits to see if a physician is integrated or not (codde list)

physician_integration_finder = function(data,
                                        integrated_place_of_service_codes = c("19", "22"),
                                        all_place_of_service_codes = c("11", "19", "22"),
                                        #integration_threshold = 0.5,
                                        office_code_list = c(
                                          "99201",
                                          "99202",
                                          "99203",
                                          "99204",
                                          "99205",
                                          "99211",
                                          "99212",
                                          "99213",
                                          "99214",
                                          "99215"
                                        ),
                                       exclusive_hospital_codes) {
  require(dtplyr)
  require(tidyverse)
  
  #data = subset(data, HCPCS_CD %in% code_list)
  result = data %>%
  mutate(
    is_facility = LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes,
    is_all = LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes,
    is_office_visit = HCPCS_CD %in% office_code_list,
    has_non_exclusive_code = HCPCS_CD %!in% exclusive_hospital_codes
  ) %>%
  group_by(PRF_PHYSN_NPI, month_year) %>%
  summarise(
    in_facility_visits_count = sum(is_facility*is_office_visit, na.rm = T),
    in_all_visits_count = sum(is_all*is_office_visit, na.rm = T),
    in_facility_non_exclusive_HCPCS_count = sum(is_facility*has_non_exclusive_code, na.rm = T),
    in_all_non_exclusive_HCPCS_count = sum(is_all*has_non_exclusive_code, na.rm = T),
    in_facility_count = sum(is_facility, na.rm = T),
    in_all_count = sum(is_all, na.rm = T)
    #tot = n(),
  ) %>%
  mutate(
    date = as.IDate(paste(month_year,"-01",sep="")),
    in_facility_visits_prp = in_facility_visits_count / in_all_visits_count,
    in_facility_non_exclusive_HCPCS_prp = in_facility_non_exclusive_HCPCS_count / in_all_non_exclusive_HCPCS_count,    
    in_facility_prp = in_facility_count / in_all_count
    #in_facility_prp_from_tot = in_facility_count / tot
    #is_integrated = in_facility_prp >= integration_threshold,
    #is_integrated_from_tot = in_facility_prp_from_tot >= integration_threshold,
  )%>%
  as.data.table()
  #result=result[,.(PRF_PHYSN_NPI,month_year,is_integrated)]
  #result=reshape(result, idvar = "PRF_PHYSN_NPI", timevar = "month_year", direction = "wide")
  #setcolorder(result,order(colnames(result)))
  #setcolorder(result,"PRF_PHYSN_NPI")  
}

physician_integration_stats = physician_integration_finder(carrier_data_all_years,exclusive_hospital_codes=exclusive_hospital_codes)

`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.


In [27]:
tail(physician_integration_stats)

PRF_PHYSN_NPI,month_year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,date,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<date>,<dbl>,<dbl>,<dbl>
9999999992,2013-05,0,0,0,1,0,4,2013-05-01,NaN,0,0
9999999992,2013-06,0,0,0,0,0,6,2013-06-01,NaN,NaN,0
9999999992,2013-07,0,0,0,0,0,0,2013-07-01,NaN,NaN,NaN
9999999992,2013-08,0,0,0,0,0,8,2013-08-01,NaN,NaN,0
9999999992,2013-09,0,0,0,0,0,0,2013-09-01,NaN,NaN,NaN
9999999992,2013-10,0,0,0,1,0,7,2013-10-01,NaN,0,0


### Add integration status of physicians at the year of diagnosis
This function will add the integration status of most common physicians to each patient's summary data.

In [31]:
#rename columns
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status=function(data, physician_integration_stats){
  
  data=data[,c("DESY_SORT_KEY",
               "first_diagnosis",
               "most_common_physician_PRF_PHYSN_NPI",
               "most_common_primary_care_physician_PRF_PHYSN_NPI",
               "most_common_cardiologist_PRF_PHYSN_NPI",
               'catheterization_doc_NPI',
               'angioplasty_doc_NPI',
               'diagnosing_doc_NPI'
              #,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
              )]
  
  most_common_physician = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_physician,
    ncol(physician_integration_stats)-1,
    paste("most_common_physician_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_physician=most_common_physician[,-c("most_common_primary_care_physician_PRF_PHYSN_NPI",
                                                  "most_common_cardiologist_PRF_PHYSN_NPI",
                                                  'catheterization_doc_NPI',
                                                  'angioplasty_doc_NPI',
                                                  'diagnosing_doc_NPI'
                                                 )]
    
    
  most_common_primary_care = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats)-1,
    paste("most_common_primary_care_physician_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_primary_care=most_common_primary_care[,-c("most_common_physician_PRF_PHYSN_NPI",
                                                        "most_common_cardiologist_PRF_PHYSN_NPI",
                                                        'catheterization_doc_NPI',
                                                        'angioplasty_doc_NPI',
                                                        'diagnosing_doc_NPI')]  
  
  most_common_cardiologist = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_cardiologist_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_cardiologist,
    ncol(physician_integration_stats)-1,
    paste("most_common_cardiologist_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_cardiologist=most_common_cardiologist[,-c("most_common_physician_PRF_PHYSN_NPI",
                                                        "most_common_primary_care_physician_PRF_PHYSN_NPI",
                                                        'catheterization_doc_NPI',
                                                        'angioplasty_doc_NPI',
                                                        'diagnosing_doc_NPI')]
  diagnosing_doc = left_join(
    data,
    physician_integration_stats,
    by = c(
      "diagnosing_doc_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    diagnosing_doc,
    ncol(physician_integration_stats)-1,
    paste("diagnosing_doc_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  diagnosing_doc=diagnosing_doc[,-c("most_common_physician_PRF_PHYSN_NPI",
                                    "most_common_cardiologist_PRF_PHYSN_NPI",
                                    "most_common_primary_care_physician_PRF_PHYSN_NPI",
                                    'catheterization_doc_NPI',
                                    'angioplasty_doc_NPI')]
                                                       
  
  catheterization_doc = left_join(
    data,
    physician_integration_stats,
    by = c(
      'catheterization_doc_NPI' = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    catheterization_doc,
    ncol(physician_integration_stats)-1,
    paste("catheterization_doc_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  catheterization_doc=catheterization_doc[,-c("most_common_physician_PRF_PHYSN_NPI",
                                                        "most_common_cardiologist_PRF_PHYSN_NPI",
                                                        "most_common_primary_care_physician_PRF_PHYSN_NPI",
                                                        'angioplasty_doc_NPI',
                                                        'diagnosing_doc_NPI')]
  
  angioplasty_doc = left_join(
    data,
    physician_integration_stats,
    by = c(
      "angioplasty_doc_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    angioplasty_doc,
    ncol(physician_integration_stats)-1,
    paste("angioplasty_doc_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  angioplasty_doc=angioplasty_doc[,-c("most_common_physician_PRF_PHYSN_NPI",
                                                        "most_common_cardiologist_PRF_PHYSN_NPI",
                                                        "most_common_primary_care_physician_PRF_PHYSN_NPI",
                                                        'catheterization_doc_NPI',
                                                        'diagnosing_doc_NPI')]
  
  
  return(list(most_common_physician,
              most_common_primary_care,
              most_common_cardiologist,
              diagnosing_doc,
              catheterization_doc,
              angioplasty_doc
             ))
}

melted_physician_integration_stats_stable_angina=add_integration_status(
  data = yearly_calculations_stable_angina,
  physician_integration_stats = physician_integration_stats)

melted_physician_integration_stats_unstable_angina=add_integration_status(
  data = yearly_calculations_unstable_angina,
  physician_integration_stats = physician_integration_stats)




In [32]:
lapply(melted_physician_integration_stats_stable_angina,head)

DESY_SORT_KEY,first_diagnosis,most_common_physician_PRF_PHYSN_NPI,most_common_physician_month_year,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_date,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp
<int>,<date>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<date>,<dbl>,<dbl>,<dbl>
100007109,2013-12-26,1922185792,2013-01,0,1,0,1,0,3,2013-01-01,0,0,0
100007109,2013-12-26,1922185792,2013-02,0,0,0,0,0,0,2013-02-01,NaN,NaN,NaN
100007109,2013-12-26,1922185792,2013-04,0,1,0,1,0,1,2013-04-01,0,0,0
100007109,2013-12-26,1922185792,2013-05,0,1,0,1,0,2,2013-05-01,0,0,0
100007109,2013-12-26,1922185792,2013-06,0,1,0,2,0,2,2013-06-01,0,0,0
100007109,2013-12-26,1922185792,2013-07,0,1,0,1,0,1,2013-07-01,0,0,0
DESY_SORT_KEY,first_diagnosis,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_month_year,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_date,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp
<int>,<date>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<date>,<dbl>,<dbl>,<dbl>
100007109,2013-12-26,1922185792,2013-01,0,1,0,1,0,3,2013-01-01,0,0,0


### Add integration status of 2013 physicians of patients (for instrumental analysis)

In [33]:
#rename columns
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status_2013=function(data, physician_integration_stats){
  
  data=data[,c("DESY_SORT_KEY"
              ,"first_diagnosis" 
              ,"most_common_physician_2013_PRF_PHYSN_NPI"
              ,"most_common_primary_care_physician_2013_PRF_PHYSN_NPI"
              ,"most_common_cardiologist_2013_PRF_PHYSN_NPI"
              #,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
              )]
  
  most_common_physician = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_physician_2013_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_physician,
    ncol(physician_integration_stats)-1,
    paste("most_common_physician_2013_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_physician=most_common_physician[,-c("most_common_primary_care_physician_2013_PRF_PHYSN_NPI"
                                                  ,"most_common_cardiologist_2013_PRF_PHYSN_NPI"
                                                  #,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
                                                 )]
    
    
  most_common_primary_care = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_2013_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats)-1,
    paste("most_common_primary_care_physician_2013_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_primary_care=most_common_primary_care[,-c("most_common_physician_2013_PRF_PHYSN_NPI"
                                                  ,"most_common_cardiologist_2013_PRF_PHYSN_NPI"
                                                  #,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
                                                       )]  
  
  most_common_cardiologist = left_join(
    data,
    physician_integration_stats,
    by = c(
      "most_common_cardiologist_2013_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    most_common_cardiologist,
    ncol(physician_integration_stats)-1,
    paste("most_common_cardiologist_2013_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
    )
  most_common_cardiologist=most_common_cardiologist[,-c("most_common_physician_2013_PRF_PHYSN_NPI"
                                                  ,"most_common_primary_care_physician_2013_PRF_PHYSN_NPI"
                                                  #,"most_common_interventional_cardiologist_PRF_PHYSN_NPI"
                                                       )]
  
  #most_common_interventional_cardiologist = left_join(
  #  data,
  #  physician_integration_stats,
  #  by = c(
  #    "most_common_interventional_cardiologist_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI")
  #) %>% as.data.table()
  #rename_last(
  #  most_common_interventional_cardiologist,
  #  ncol(physician_integration_stats)-1,
  #  paste("most_common_interventional_cardiologist_",colnames(physician_integration_stats)[2:ncol(physician_integration_stats)],sep="")
  #  )
  #most_common_interventional_cardiologist=most_common_interventional_cardiologist[,-c("most_common_physician_PRF_PHYSN_NPI"
  #                                                ,"most_common_primary_care_physician_PRF_PHYSN_NPI"
  #                                                ,"most_common_cardiologist_PRF_PHYSN_NPI")]
  
  return(list(most_common_physician,
              most_common_primary_care,
              most_common_cardiologist
              #most_common_interventional_cardiologist
             ))
}

melted_physician_integration_stats_2013_stable_angina=add_integration_status_2013(
  data = yearly_calculations_stable_angina,
  physician_integration_stats = physician_integration_stats)

melted_physician_integration_stats_2013_unstable_angina=add_integration_status_2013(
  data = yearly_calculations_unstable_angina,
  physician_integration_stats = physician_integration_stats)


In [34]:
lapply(melted_physician_integration_stats_2013_stable_angina,head)

DESY_SORT_KEY,first_diagnosis,most_common_physician_2013_PRF_PHYSN_NPI,most_common_physician_2013_month_year,most_common_physician_2013_in_facility_visits_count,most_common_physician_2013_in_all_visits_count,most_common_physician_2013_in_facility_non_exclusive_HCPCS_count,most_common_physician_2013_in_all_non_exclusive_HCPCS_count,most_common_physician_2013_in_facility_count,most_common_physician_2013_in_all_count,most_common_physician_2013_date,most_common_physician_2013_in_facility_visits_prp,most_common_physician_2013_in_facility_non_exclusive_HCPCS_prp,most_common_physician_2013_in_facility_prp
<int>,<date>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<date>,<dbl>,<dbl>,<dbl>
100007109,2013-12-26,1922185792,2013-01,0,1,0,1,0,3,2013-01-01,0,0,0
100007109,2013-12-26,1922185792,2013-02,0,0,0,0,0,0,2013-02-01,NaN,NaN,NaN
100007109,2013-12-26,1922185792,2013-04,0,1,0,1,0,1,2013-04-01,0,0,0
100007109,2013-12-26,1922185792,2013-05,0,1,0,1,0,2,2013-05-01,0,0,0
100007109,2013-12-26,1922185792,2013-06,0,1,0,2,0,2,2013-06-01,0,0,0
100007109,2013-12-26,1922185792,2013-07,0,1,0,1,0,1,2013-07-01,0,0,0
DESY_SORT_KEY,first_diagnosis,most_common_primary_care_physician_2013_PRF_PHYSN_NPI,most_common_primary_care_physician_2013_month_year,most_common_primary_care_physician_2013_in_facility_visits_count,most_common_primary_care_physician_2013_in_all_visits_count,most_common_primary_care_physician_2013_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_2013_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_2013_in_facility_count,most_common_primary_care_physician_2013_in_all_count,most_common_primary_care_physician_2013_date,most_common_primary_care_physician_2013_in_facility_visits_prp,most_common_primary_care_physician_2013_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_2013_in_facility_prp
<int>,<date>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<date>,<dbl>,<dbl>,<dbl>
100007109,2013-12-26,1922185792,2013-01,0,1,0,1,0,3,2013-01-01,0,0,0


In [35]:
head(yearly_calculations_stable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

### Finding the changes in integration status of physicians during the year of diagnosis

#### In all codes

#### Vistist only 

#### Non-exclusive HCPCS codes  

In [58]:
change_stats_summarizer_non_exclusive_only = function(data,melted_physician_integration_stats,specialty){

  require(dtplyr)
  require(lubridate)
  require(tidyverse)

  looking_for=paste(specialty,"_PRF_PHYSN_NPI",sep="")
  date_looking_for=paste(specialty,"_date",sep="")
  in_facility_prp_looking_for=paste(specialty,"_in_facility_non_exclusive_HCPCS_prp",sep="")
  in_all_count_looking_for=paste(specialty,"_in_all_non_exclusive_HCPCS_count",sep="")
  in_facility_count_looking_for=paste(specialty,"_in_facility_non_exclusive_HCPCS_count",sep="")
  
  
  result=melted_physician_integration_stats%>%
  group_by(DESY_SORT_KEY)%>%
  mutate(n_integration_stat_year_of_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_2013=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31") &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_year_before_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
           first_diagnosis-!!rlang::sym(date_looking_for)<=365 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))]
         
        )%>%
  
  summarise(prp_in_year_of_diagnosis_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5 , na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_05_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
         prp_in_year_of_diagnosis_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_03_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
        avg_in_year_of_diagnosis_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],

         
         avg_in_2013_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],
         
         avg_in_year_before_diagnosis_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)]

        )%>%
  distinct()%>%
  as.data.table()
  
  #rename last n columns in a dataset
  rename_last = function(data, how_many, new_names) {
    total_cols = ncol(data)
    setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
  }
  
  rename_last(
    result,
    9,
    paste(specialty,colnames(result)[(ncol(result)-8):ncol(result)],sep="_")
    )
  
  data = left_join(
    data,
    result,
    by = "DESY_SORT_KEY"
  ) %>% as.data.table()
  
  return(data)
  
}

change_stats_summarized_non_exclusive_HCPCS=change_stats_summarizer_non_exclusive_only(yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[3]],"most_common_cardiologist")


#### For 2013 (instrumental variable)

In [59]:
change_stats_summarizer_non_exclusive_only_2013 = function(data,melted_physician_integration_stats_2013,specialty){

  require(dtplyr)
  require(lubridate)
  require(tidyverse)

  looking_for=paste(specialty,"_2013_PRF_PHYSN_NPI",sep="")
  date_looking_for=paste(specialty,"_2013_date",sep="")
  in_facility_prp_looking_for=paste(specialty,"_2013_in_facility_non_exclusive_HCPCS_prp",sep="")
  in_all_count_looking_for=paste(specialty,"_2013_in_all_non_exclusive_HCPCS_count",sep="")
  in_facility_count_looking_for=paste(specialty,"_2013_in_facility_non_exclusive_HCPCS_count",sep="")
  
  
  result=melted_physician_integration_stats_2013%>%
  group_by(DESY_SORT_KEY)%>%
  mutate(n_integration_stat_year_of_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_2013=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31") &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))],
         
         n_integration_stat_year_before_diagnosis=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
           first_diagnosis-!!rlang::sym(date_looking_for)<=365 &
           !is.na(!!rlang::sym(in_facility_prp_looking_for)) ,
           length(!!rlang::sym(in_facility_prp_looking_for))]
         
        )%>%
  
  summarise(prp_in_year_of_diagnosis_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5 , na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_05_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_05_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.5, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
         prp_in_year_of_diagnosis_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_of_diagnosis,
         
         prp_in_2013_03_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_2013,
         
         prp_in_year_before_diagnosis_03_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_prp_looking_for)>=0.3, na.rm=T)]/
         n_integration_stat_year_before_diagnosis,
         
        avg_in_year_of_diagnosis_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)-first_diagnosis<=365 &
           !!rlang::sym(date_looking_for)-first_diagnosis>=0,
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],

         
         avg_in_2013_non_exclusive_HCPCS=
         .[!!rlang::sym(date_looking_for)<as.IDate("2013-12-31"),
           sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)],
         
         avg_in_year_before_diagnosis_non_exclusive_HCPCS=
            .[!!rlang::sym(date_looking_for)-first_diagnosis<=0 &
              first_diagnosis-!!rlang::sym(date_looking_for)<=365,
              sum(!!rlang::sym(in_facility_count_looking_for), na.rm=T) / sum(!!rlang::sym(in_all_count_looking_for), na.rm=T)]

        )%>%
  distinct()%>%
  as.data.table()
  
  #rename last n columns in a dataset
  rename_last = function(data, how_many, new_names) {
    total_cols = ncol(data)
    setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
  }
  
  rename_last(
    result,
    9,
    paste(specialty,"2013",colnames(result)[(ncol(result)-8):ncol(result)],sep="_")
    )
  
  data = left_join(
    data,
    result,
    by = "DESY_SORT_KEY"
  ) %>% as.data.table()
  
  return(data)
  
}

change_stats_summarized_non_exclusive_HCPCS=change_stats_summarizer_non_exclusive_only_2013(change_stats_summarized_non_exclusive_HCPCS,melted_physician_integration_stats_2013_stable_angina[[3]],"most_common_cardiologist")


In [60]:
tail(change_stats_summarized_non_exclusive_HCPCS)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

#### Adding integration stats to data

In [61]:
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[1]],"most_common_physician")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[2]],"most_common_primary_care_physician")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[3]],"most_common_cardiologist")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[4]],"diagnosing_doc")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[5]],"catheterization_doc")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_stable_angina,melted_physician_integration_stats_stable_angina[[6]],"angioplasty_doc")

yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[1]],"most_common_physician")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[2]],"most_common_primary_care_physician")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[3]],"most_common_cardiologist")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[4]],"diagnosing_doc")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[5]],"catheterization_doc")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_unstable_angina[[6]],"angioplasty_doc")


In [62]:
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_stable_angina,melted_physician_integration_stats_2013_stable_angina[[1]],"most_common_physician")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_stable_angina,melted_physician_integration_stats_2013_stable_angina[[2]],"most_common_primary_care_physician")
yearly_calculations_stable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_stable_angina,melted_physician_integration_stats_2013_stable_angina[[3]],"most_common_cardiologist")

yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_2013_unstable_angina[[1]],"most_common_physician")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_2013_unstable_angina[[2]],"most_common_primary_care_physician")
yearly_calculations_unstable_angina=change_stats_summarizer_non_exclusive_only_2013(
  yearly_calculations_unstable_angina,melted_physician_integration_stats_2013_unstable_angina[[3]],"most_common_cardiologist")


In [63]:
head(yearly_calculations_stable_angina)
head(yearly_calculations_unstable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,catheterization_date,angioplasty_date,catheterization_doc_NPI,angioplasty_doc_NPI,diagnosing_doc_NPI,year_first_diagnosed,time_difference_angio_cath,angioplasty_with_cath,angioplasty_during_year_after_cath,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,tot_cheap_prcdr_count,tot_expensive_prcdr_count,tot_cheap_prcdr_count_cardiology_related,tot_expensive_prcdr_count_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICA

### Saving the results.

In [22]:
write_fst(yearly_calculations_stable_angina,
          "results_apr/yearly_calculations_stable_angina_with_integration.fst") 
write_fst(yearly_calculations_unstable_angina,
          "results_apr/yearly_calculations_unstable_angina_with_integration.fst")
write_fst(physician_integration_stats,
          "results_apr/physician_integration_stats.fst")

#I am also saving the csv to be able to share the results with colleagues later.
write.csv(yearly_calculations_stable_angina,
          "results_apr/yearly_calculations_stable_angina_with_integration.csv") 
write.csv(yearly_calculations_unstable_angina,
          "results_apr/yearly_calculations_unstable_angina_with_integration.csv")



### Finding physician transition dates

We should first convert the data to long format and add a column with the commmulative sums. We also convert the colnames to dates.

In [30]:
melted_physician_integration_stats = 
melt(physician_integration_stats, "PRF_PHYSN_NPI") %>% 
  as.data.table() %>% 
  arrange(PRF_PHYSN_NPI) %>%
  as.data.table()
melted_physician_integration_stats[is.na(value),value:=FALSE]
melted_physician_integration_stats[,date := as.IDate(paste(substr(variable,start = 15,stop=21),"-01",sep=""),format="%Y-%m-%d")]
melted_physician_integration_stats[,cumsum := cumsum(value),by=PRF_PHYSN_NPI]

PRF_PHYSN_NPI,variable,value,date,cumsum
<chr>,<fct>,<lgl>,<date>,<int>
,is_integrated.2013-01,FALSE,2013-01-01,0
,is_integrated.2013-02,FALSE,2013-02-01,0
,is_integrated.2013-03,FALSE,2013-03-01,0
,is_integrated.2013-04,FALSE,2013-04-01,0
,is_integrated.2013-05,FALSE,2013-05-01,0
,is_integrated.2013-06,FALSE,2013-06-01,0
,is_integrated.2013-07,FALSE,2013-07-01,0
,is_integrated.2013-08,FALSE,2013-08-01,0
,is_integrated.2013-09,FALSE,2013-09-01,0


We then find the dates of switching to and from integration

In [32]:
physician_integration_changes_finder= function(data) {
  
  result=data%>%
  group_by(PRF_PHYSN_NPI)%>%
  arrange(variable)%>%
  mutate(switch_to_integration=(cumsum==1 & lead(cumsum,11)==12 & lag(cumsum,1) == 0))%>%
  mutate(switch_back_to_independence=(value==FALSE & cumsum(switch_to_integration)==1 
                                 & lead(cumsum,11)-cumsum == 0 & lag(value)-value==1),
         switch_to_independece=(value==FALSE & lead(cumsum,11)-cumsum == 0 
                                & lag(value)-value==1 & .[12,cumsum]==12),
        )%>%
  arrange(PRF_PHYSN_NPI) %>%
  summarise(date_switched_to_integration=.[switch_to_integration==T,date],
            date_switched_back_to_independence=.[switch_back_to_independence==T,date],
            date_switched_to_independence=.[switch_to_independece==T,date]         
           )%>%
  as.data.table()
}
physician_integration_changes=physician_integration_changes_finder(melted_physician_integration_stats)


Warning message in `[.data.table`(`_DT13`[order(variable)][, `:=`(switch_to_integration = (cumsum == :
“Item 3 of j's result for group 1006 is zero length. This will be filled with 3 NAs to match the longest column in this result. Later groups may have a similar problem but only the first is reported to save filling the warning buffer.”


In [39]:
head(physician_integration_changes)

PRF_PHYSN_NPI,date_switched_to_integration,date_switched_back_to_independence,date_switched_to_independence
<chr>,<date>,<date>,<date>
1003000522,NA,NA,2016-07-01
1003004326,2013-08-01,2016-04-01,NA
1003012642,2015-05-01,2016-07-01,NA
1003013756,NA,NA,2015-05-01
1003015025,NA,NA,2019-08-01
1003027764,2019-09-01,NA,NA


#### Adding integration change dates to the data

In [ ]:
#adding integration changes
add_integration_switches=function(data, physician_integration_changes){
  
  #rename last n columns in a dataset
  rename_last = function(data, how_many, new_names) {
    total_cols = ncol(data)
    setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
  }
  data = left_join(
    data,
    physician_integration_changes,
    by = c("most_common_physician_PRF_PHYSN_NPI"="PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    data,
    3,
    paste("most_common_physician_",colnames(data)[(ncol(data)-2):ncol(data)],sep="")
    )
  
  data = left_join(
    data,
    physician_integration_changes,
    by = c("most_common_primary_care_physician_PRF_PHYSN_NPI"="PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    data,
    3,
    paste("most_common_primary_care_physician_",colnames(data)[(ncol(data)-2):ncol(data)],sep="")
    )
  
  data = left_join(
    data,
    physician_integration_changes,
    by = c("most_common_cardiologist_PRF_PHYSN_NPI"="PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    data,
    3,
    paste("most_common_cardiologist_",colnames(data)[(ncol(data)-2):ncol(data)],sep="")
    )
  
  data = left_join(
    data,
    physician_integration_changes,
    by = c("most_common_interventional_cardiologist_PRF_PHYSN_NPI"="PRF_PHYSN_NPI")
  ) %>% as.data.table()
  rename_last(
    data,
    3,
    paste("most_common_interventional_cardiologist_",colnames(data)[(ncol(data)-2):ncol(data)],sep="")
    )
}

yearly_calculations_stable_angina=add_integration_switches(yearly_calculations_stable_angina,physician_integration_changes)
yearly_calculations_unstable_angina=add_integration_switches(yearly_calculations_unstable_angina,physician_integration_changes)


## Comparing interventional and medical cardiologists

In [6]:
yearly_calculations_stable_angina =
read_fst("results_apr/yearly_calculations_stable_angina_with_integration.fst"
         ,as.data.table = T) 
yearly_calculations_unstable_angina =
read_fst("results_apr/yearly_calculations_unstable_angina_with_integration.fst"
         ,as.data.table = T)

In [18]:
data_for_comparison_stable_angina=
inner_join(yearly_calculations_stable_angina,yearly_patient_conditions_carrier,by = "DESY_SORT_KEY") %>% as.data.table

data_for_comparison_unstable_angina=
inner_join(yearly_calculations_unstable_angina,yearly_patient_conditions_carrier,by = "DESY_SORT_KEY") %>% as.data.table


In [19]:
head(data_for_comparison_stable_angina)

DESY_SORT_KEY,first_diagnosis,tot_allowed_carrier,office_visit_count,office_visit_cost_carrier,cardiology_visit_count,distinct_clinicians,distinct_cardiologists,distinct_primary_care_physicians,distinct_surgeons,distinct_other_specialties,catheterization_count,catheterization_cost_carrier,ecg_count,ecg_cost_carrier,cardiac_ct_count,cardiac_ct_cost_carrier,cardiac_mri_count,cardiac_mri_cost_carrier,stress_test_count,stress_test_cost_carrier,echocardiography_count,echocardiography_cost_carrier,angioplasty_count,angioplasty_cost_carrier,CABG_count,CABG_cost_carrier,stable_angina,unstable_angina,MI,cardiac_arrest,stroke,hypertension,copd,obesity,depression,diabetes,icd_9_pure,icd_10_pure,year_first_diagnosed,tot_allowed_carrier_cardiology_related,office_visit_count_cardiology_related,office_visit_cost_carrier_cardiology_related,cardiology_visit_count_cardiology_related,distinct_clinicians_cardiology_related,distinct_cardiologists_cardiology_related,distinct_primary_care_physicians_cardiology_related,distinct_surgeons_cardiology_related,distinct_other_specialties_cardiology_related,catheterization_count_cardiology_related,catheterization_cost_carrier_cardiology_related,ecg_count_cardiology_related,ecg_cost_carrier_cardiology_related,cardiac_ct_count_cardiology_related,cardiac_ct_cost_carrier_cardiology_related,cardiac_mri_count_cardiology_related,cardiac_mri_cost_carrier_cardiology_related,stress_test_count_cardiology_related,stress_test_cost_carrier_cardiology_related,echocardiography_count_cardiology_related,echocardiography_cost_carrier_cardiology_related,angioplasty_count_cardiology_related,angioplasty_cost_carrier_cardiology_related,CABG_count_cardiology_related,CABG_cost_carrier_cardiology_related,office_visit_cost_outpatient,catheterization_cost_outpatient,ecg_cost_outpatient,cardiac_ct_cost_outpatient,cardiac_mri_cost_outpatient,stress_test_cost_outpatient,echocardiography_cost_outpatient,angioplasty_cost_outpatient,CABG_cost_outpatient,office_visit_cost_outpatient_cardiology_related,catheterization_cost_outpatient_cardiology_related,ecg_cost_outpatient_cardiology_related,cardiac_ct_cost_outpatient_cardiology_related,cardiac_mri_cost_outpatient_cardiology_related,stress_test_cost_outpatient_cardiology_related,echocardiography_cost_outpatient_cardiology_related,angioplasty_cost_outpatient_cardiology_related,CABG_cost_outpatient_cardiology_related,office_visit_cost,catheterization_cost,ecg_cost,cardiac_ct_cost,cardiac_mri_cost,stress_test_cost,echocardiography_cost,angioplasty_cost,CABG_cost,office_visit_cost_cardiology_related,catheterization_cost_cardiology_related,ecg_cost_cardiology_related,cardiac_ct_cost_cardiology_related,cardiac_mri_cost_cardiology_related,stress_test_cost_cardiology_related,echocardiography_cost_cardiology_related,angioplasty_cost_cardiology_related,CABG_cost_cardiology_related,tot_cheap_prcdr_cost,tot_expensive_prcdr_cost,tot_cheap_prcdr_cost_cardiology_related,tot_expensive_prcdr_cost_cardiology_related,REFERENCE_YEAR_2013,STATE_CODE_2013,COUNTY_CODE_2013,SEX_CODE_2013,RACE_CODE_2013,AGE_2013,ORIG_REASON_FOR_ENTITLEMENT_2013,CURR_REASON_FOR_ENTITLEMENT_2013,ENTITLEMENT_BUY_IN_IND01_2013,ENTITLEMENT_BUY_IN_IND02_2013,ENTITLEMENT_BUY_IN_IND03_2013,ENTITLEMENT_BUY_IN_IND04_2013,ENTITLEMENT_BUY_IN_IND05_2013,ENTITLEMENT_BUY_IN_IND06_2013,ENTITLEMENT_BUY_IN_IND07_2013,ENTITLEMENT_BUY_IN_IND08_2013,ENTITLEMENT_BUY_IN_IND09_2013,ENTITLEMENT_BUY_IN_IND10_2013,ENTITLEMENT_BUY_IN_IND11_2013,ENTITLEMENT_BUY_IN_IND12_2013,HMO_INDICATOR01_2013,HMO_INDICATOR02_2013,HMO_INDICATOR03_2013,HMO_INDICATOR04_2013,HMO_INDICATOR05_2013,HMO_INDICATOR06_2013,HMO_INDICATOR07_2013,HMO_INDICATOR08_2013,HMO_INDICATOR09_2013,HMO_INDICATOR10_2013,HMO_INDICATOR11_2013,HMO_INDICATOR12_2013,VALID_DATE_OF_DEATH_SWITCH_2013,DATE_OF_DEATH_2013,REFERENCE_YEAR_2014,STATE_CODE_2014,COUNTY_CODE_2014,SEX_CODE_2014,RACE_CODE_2014,AGE_2014,ORIG_REASON_FOR_ENTITLEMENT_2014,CURR_REASON_FOR_ENTITLEMENT_2014,ENTITLEMENT_BUY_IN_IND01_2014,ENTITLE

In [115]:
compare_interventional_to_medical=function(data){
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  
  data%>%
  group_by(DESY_SORT_KEY)%>%
  filter(date - first_diagnosis >= 0 &
           date - first_diagnosis < 365,
         .preserve = T) %>%  
  summarise(
    
    n_service_by_medical=nrow(.[PRVDR_SPCLTY=="06"]),
    n_service_by_interventional=nrow(.[PRVDR_SPCLTY=="C3"]),
    
    n_visit_by_medical=nrow(.[PRVDR_SPCLTY=="06" & is_office_visit]),
    n_visit_by_interventional=nrow(.[PRVDR_SPCLTY=="C3" & is_office_visit]),
    
    prp_service_by_medical=nrow(.[PRVDR_SPCLTY=="06"])/n(),
    prp_service_by_interventional=nrow(.[PRVDR_SPCLTY=="C3"])/n(),
    
    prp_visit_by_medical=nrow(.[PRVDR_SPCLTY=="06" & is_office_visit])/nrow(.[is_office_visit]),
    prp_visit_by_interventional=nrow(.[PRVDR_SPCLTY=="C3" & is_office_visit])/nrow(.[is_office_visit]),
    
    n_service_by_dominant_medical=nrow(.[PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    n_service_by_dominant_interventional=
      nrow(.[PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    
    n_visit_by_dominant_medical=
      nrow(.[PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    n_visit_by_dominant_interventional=
      nrow(.[PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    
    prp_service_by_dominant_medical=
      nrow(.[PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/n(),
    prp_service_by_dominant_interventional=
      nrow(.[PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/n(),
    
    prp_visit_by_dominant_medical=
      nrow(.[PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_office_visit]),
    prp_visit_by_dominant_interventional=
      nrow(.[PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_office_visit]),
    
    prp_service_by_dominant_medical_within_cardiologists=
      nrow(.[PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/nrow(.[PRVDR_SPCLTY %in% c("06","C3")]),
    prp_service_by_dominant_interventional_within_cardiologists=
      nrow(.[PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/nrow(.[PRVDR_SPCLTY %in% c("06","C3")]),
    
    prp_visit_by_dominant_medical_within_cardiologists=
      nrow(.[PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_office_visit & PRVDR_SPCLTY %in% c("06","C3")]),
    prp_visit_by_dominant_interventional_within_cardiologists=
      nrow(.[PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_office_visit & PRVDR_SPCLTY %in% c("06","C3")]),
    
    distinct_medical_all_services=nrow(data.frame(.[PRVDR_SPCLTY=="06", unique(PRF_PHYSN_NPI)])),
    distinct_interventional_all_services=nrow(data.frame(.[PRVDR_SPCLTY=="C3", unique(PRF_PHYSN_NPI)])),
    distinct_medical_visits=nrow(data.frame(.[PRVDR_SPCLTY=="06" & is_office_visit , unique(PRF_PHYSN_NPI)])),
    distinct_interventional_all_visits=nrow(data.frame(.[PRVDR_SPCLTY=="C3" & is_office_visit , unique(PRF_PHYSN_NPI)])),
    
    has_both_medical_and_interventional=nrow(.[PRVDR_SPCLTY=="06"])>0 & nrow(.[PRVDR_SPCLTY=="C3"])>0,
    only_has_medical=nrow(.[PRVDR_SPCLTY=="06"])>0 & nrow(.[PRVDR_SPCLTY=="C3"])==0,
    only_has_interventional=nrow(.[PRVDR_SPCLTY=="06"])==0 & nrow(.[PRVDR_SPCLTY=="C3"])>0,
    has_no_medical_no_interventional=nrow(.[PRVDR_SPCLTY=="06"])==0 & nrow(.[PRVDR_SPCLTY=="C3"])==0,
    
    n_service_by_medical_cardiology_related=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06"]),
    n_service_by_interventional_cardiology_related=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3"]),
    
    n_visit_by_medical_cardiology_related=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit]),
    n_visit_by_interventional_cardiology_related=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit]),
    
    n_service_by_medical_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06"]),
    n_service_by_interventional_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3"]),
    
    n_visit_by_medical_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit]),
    n_visit_by_interventional_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit]),
    
    prp_service_by_medical_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06"])/n(),
    prp_service_by_interventional_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3"])/n(),
    
    prp_visit_by_medical_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit])/nrow(.[is_cardiology_related & is_office_visit]),
    prp_visit_by_interventional_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit])/nrow(.[is_cardiology_related & is_office_visit]),
    
    n_service_by_dominant_medical_cardiology_related_=nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    n_service_by_dominant_interventional_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    
    n_visit_by_dominant_medical_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    n_visit_by_dominant_interventional_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI]),
    
    prp_service_by_dominant_medical_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/n(),
    prp_service_by_dominant_interventional_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/n(),
    
    prp_visit_by_dominant_medical_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_cardiology_related & is_office_visit]),
    prp_visit_by_dominant_interventional_cardiology_related_=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_cardiology_related & is_office_visit]),
    
    prp_service_by_dominant_medical_cardiology_related__within_cardiologists=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/nrow(.[is_cardiology_related & PRVDR_SPCLTY %in% c("06","C3")]),
    prp_service_by_dominant_interventional_cardiology_related__within_cardiologists=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/nrow(.[is_cardiology_related & PRVDR_SPCLTY %in% c("06","C3")]),
    
    prp_visit_by_dominant_medical_cardiology_related__within_cardiologists=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="06" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_cardiology_related & is_office_visit & PRVDR_SPCLTY %in% c("06","C3")]),
    prp_visit_by_dominant_interventional_cardiology_related__within_cardiologists=
      nrow(.[is_cardiology_related & PRVDR_SPCLTY=="C3" & is_office_visit & PRF_PHYSN_NPI == most_common_cardiologist_PRF_PHYSN_NPI])/
      nrow(.[is_cardiology_related & is_office_visit & PRVDR_SPCLTY %in% c("06","C3")])
  )%>%
  as.data.table()
}

In [116]:
interventional_vs_medical_stable_angine=compare_interventional_to_medical(data=data_for_comparison_stable_angina)
interventional_vs_medical_unstable_angine=compare_interventional_to_medical(data=data_for_comparison_unstable_angina)

In [119]:
head(interventional_vs_medical_unstable_angine)

DESY_SORT_KEY,n_service_by_medical,n_service_by_interventional,n_visit_by_medical,n_visit_by_interventional,prp_service_by_medical,prp_service_by_interventional,prp_visit_by_medical,prp_visit_by_interventional,n_service_by_dominant_medical,n_service_by_dominant_interventional,n_visit_by_dominant_medical,n_visit_by_dominant_interventional,prp_service_by_dominant_medical,prp_service_by_dominant_interventional,prp_visit_by_dominant_medical,prp_visit_by_dominant_interventional,prp_service_by_dominant_medical_within_cardiologists,prp_service_by_dominant_interventional_within_cardiologists,prp_visit_by_dominant_medical_within_cardiologists,prp_visit_by_dominant_interventional_within_cardiologists,distinct_medical_all_services,distinct_interventional_all_services,distinct_medical_visits,distinct_interventional_all_visits,has_both_medical_and_interventional,only_has_medical,only_has_interventional,has_no_medical_no_interventional,n_service_by_medical_cardiology_related,n_service_by_interventional_cardiology_related,n_visit_by_medical_cardiology_related,n_visit_by_interventional_cardiology_related,n_service_by_medical_cardiology_related_,n_service_by_interventional_cardiology_related_,n_visit_by_medical_cardiology_related_,n_visit_by_interventional_cardiology_related_,prp_service_by_medical_cardiology_related_,prp_service_by_interventional_cardiology_related_,prp_visit_by_medical_cardiology_related_,prp_visit_by_interventional_cardiology_related_,n_service_by_dominant_medical_cardiology_related_,n_service_by_dominant_interventional_cardiology_related_,n_visit_by_dominant_medical_cardiology_related_,n_visit_by_dominant_interventional_cardiology_related_,prp_service_by_dominant_medical_cardiology_related_,prp_service_by_dominant_interventional_cardiology_related_,prp_visit_by_dominant_medical_cardiology_related_,prp_visit_by_dominant_interventional_cardiology_related_,prp_service_by_dominant_medical_cardiology_related__within_cardiologists,prp_service_by_dominant_interventional_cardiology_related__within_cardiologists,prp_visit_by_dominant_medical_cardiology_related__within_cardiologists,prp_visit_by_dominant_interventional_cardiology_related__within_cardiologists
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
100006149,7,0,1,0,0.17073171,0,0.10000000,0,7,0,1,0,0.17073171,0,0.10000000,0,1.00,0,1,0,1,0,1,0,FALSE,TRUE,FALSE,FALSE,7,0,1,0,7,0,1,0,0.17073171,0,0.5,0,7,0,1,0,0.17073171,0,0.5,0,1.0000000,0,1,0
100010573,5,0,0,0,0.06578947,0,0.00000000,0,0,0,0,0,0.00000000,0,0.00000000,0,0.00,0,NaN,NaN,3,0,0,0,FALSE,TRUE,FALSE,FALSE,4,0,0,0,4,0,0,0,0.05263158,0,0.0,0,0,0,0,0,0.00000000,0,0.0,0,0.0000000,0,NaN,NaN
100012125,4,0,1,0,0.12903226,0,0.06666667,0,3,0,1,0,0.09677419,0,0.06666667,0,0.75,0,1,0,2,0,1,0,FALSE,TRUE,FALSE,FALSE,3,0,1,0,3,0,1,0,0.09677419,0,1.0,0,2,0,1,0,0.06451613,0,1.0,0,0.6666667,0,1,0
100029887,5,0,0,0,0.08620690,0,0.00000000,0,0,0,0,0,0.00000000,0,0.00000000,0,0.00,0,NaN,NaN,4,0,0,0,FALSE,TRUE,FALSE,FALSE,4,0,0,0,4,0,0,0,0.06896552,0,0.0,0,0,0,0,0,0.00000000,0,0.0,0,0.0000000,0,NaN,NaN
100035075,5,0,0,0,0.07462687,0,0.00000000,0,0,0,0,0,0.00000000,0,0.00000000,0,0.00,0,NaN,NaN,2,0,0,0,FALSE,TRUE,FALSE,FALSE,0,0,0,0,0,0,0,0,0.00000000,0,NaN,NaN,0,0,0,0,0.00000000,0,NaN,NaN,NaN,NaN,NaN,NaN
100041765,10,0,1,0,0.50000000,0,0.33333333,0,6,0,1,0,0.30000000,0,0.33333333,0,0.60,0,1,0,4,0,1,0,FALSE,TRUE,FALSE,FALSE,1,0,0,0,1,0,0,0,0.05000000,0,NaN,NaN,0,0,0,0,0.00000000,0,NaN,NaN,0.0000000,0,NaN,NaN


In [ ]:
write_fst(interventional_vs_medical_stable_angina,"interventional_vs_medical_stable_angina.fst")
write_fst(interventional_vs_medical_unstable_angina,"interventional_vs_medical_unstable_angina.fst")